In [1]:
import os
if not os.environ.get("PREAMBLE_RUN", False): 
    %run -i "../preamble.py" 2

In [2]:
import pandas as pd
import numpy as np
import os

from src.constants import PATH_ORIGINAL_CHES_RAW_CSV, PATH_INTERMED_CHES_50_CHES_META, PATH_INTERMED_CHES_50_CHES_RAW

# set display options for the notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Since the (raw) CHES dataset only includes country ID, party ID and party label per national party/row/entry and we do not have a mapping from the CHES party IDs to the party IDs of the European Open Data Portal, we need more meta data on the CHES entries to infer a valid mapping. Unfortunately, we were not able to find an official database for the accompanying meta data of CHES but only a table of meta data in the Codebook.pdf [see here](https://www.chesdata.eu/s/1999-2024_CHES_codebook.pdf).
Some public repositories did their own parsing of the file but these were mainly not up to date so we will undertake the parsing ourselves.

In [3]:
# copy the mapping of country codes and country IDs from the codebook

# mapping of CHES country codes
country_codes_CHES = {
    "BE"    : 1 ,   # Belgium               'BEL'
    "DK"    : 2 ,   # Denmark               'DNK'
    "GE"    : 3 ,   # Germany               'DEU'
    "GR"    : 4 ,   # Greece                'GRC'
    "ESP"   : 5 ,   # Spain                 'ESP'
    "FR"    : 6 ,   # France                'FRA'
    "IRL"   : 7 ,   # Ireland               'IRL'
    "IT"    : 8 ,   # Italy                 'ITA'
    "NL"    : 10,   # Netherlands           'NLD'
    "UK"    : 11,   # United Kingdom        'GBR'
    "POR"   : 12,   # Portugal              'PRT'
    "AUS"   : 13,   # Austria               'AUT'
    "FIN"   : 14,   # Finland               'FIN'
    "SV"    : 16,   # Sweden                'SWE'
    "BUL"   : 20,   # Bulgaria              'BGR'
    "CZECH" : 21,   # Czech Republic        'CZE'
    "EST"   : 22,   # Estonia               'EST'
    "HUNG"  : 23,   # Hungary               'HUN'
    "LAT"   : 24,   # Latvia                'LVA'
    "LITH"  : 25,   # Lithuania             'LTU'
    "POL"   : 26,   # Poland                'POL'
    "ROM"   : 27,   # Romania               'ROU'
    "SLO"   : 28,   # Slovakia              'SVK'
    "SLE"   : 29,   # Slovenia              'SVN'
    "CRO"   : 31,   # Croatia               'HRV'
    "MAL"   : 37,   # Malta                 'MLT'
    "LUX"   : 38,   # Luxembourg            'LUX'
    "CYP"   : 40    # Cyprus                'CYP'
}

The country codes used by the EU OpenData Portal are of ISO 3166 Alpha-3 standard, so lets define a mapping to these too

In [4]:
# mapping to ISO country codes (ISO 3166 Alpha-3; used by EU OpenData Portal)
country_codes_ISO = {
    1   :   'BEL',   # Belgium
    2   :   'DNK',   # Denmark
    3   :   'DEU',   # Germany
    4   :   'GRC',   # Greece
    5   :   'ESP',   # Spain
    6   :   'FRA',   # France
    7   :   'IRL',   # Ireland
    8   :   'ITA',   # Italy
    10  :   'NLD',   # Netherlands
    11  :   'GBR',   # United Kingdom
    12  :   'PRT',   # Portugal
    13  :   'AUT',   # Austria
    14  :   'FIN',   # Finland
    16  :   'SWE',   # Sweden
    20  :   'BGR',   # Bulgaria
    21  :   'CZE',   # Czech Republic
    22  :   'EST',   # Estonia
    23  :   'HUN',   # Hungary
    24  :   'LVA',   # Latvia
    25  :   'LTU',   # Lithuania
    26  :   'POL',   # Poland
    27  :   'ROU',   # Romania
    28  :   'SVK',   # Slovakia
    29  :   'SVN',   # Slovenia
    31  :   'HRV',   # Croatia
    37  :   'MLT',   # Malta
    38  :   'LUX',   # Luxembourg
    40  :   'CYP'    # Cyprus
}

The codebook contains tables for all the national parties included in CHES, each entry with their country code, ches party ID, party label, name (native) and english name as well as variants thereof. Since it also includes various special characters, multiple parallel columns without clean separation and with different row heights here and there, as well as multi-entry cells and very tight spacing occasionally -> OCR approaches proved to be unreliable!
Therefore we will parse the tables, page by page using an external LLM (ChatGPT 5.2) with the following command prompt:
"Can you read in a picture for me and manually transcribe it? It's a table with the columns "Country", "Party ID", "Party Abbrev", "Party Name", and "Party Name (English)" and has various special characters to it as well as multiple parallel columns with different row heights here then there, serif fonts, very tight spacing occasionally, multi-entry cells (e.g., "VLD; Open VLD; PVV") and Hard line breaks inside cells. This all makes OCR highly unreliable. In the end, I need you to convert the table into a pandas dataframe. Could you do that for me?" It created lists of lists and established the following parsing rules: 
- diacritics preserve
- multi-line cells → multiple entries (e.g. party labels) for the same party
- semicolons / hard line breaks → list entries as well
- '/' and '–' preserved inside single entries

The results were manually checked for consistency and correctness (in particular, that the ID and party label(s) were correctly parsed)

In [5]:
# setup for final dataframe creation
columns = [
    "Country",
    "Party ID",
    "Party Abbrev",
    "Party Name",
    "Party Name (English)",
]

all_party_tables = []

In [6]:
# insert the parsing results from GPT
# Belgian parties
parties_BE = [
    ["BE", 102,
     ["PS"],
     ["Parti Socialiste"],
     ["Socialist Party"]],

    ["BE", 103,
     ["Vooruit/SP", "SPA"],
     ["Vooruit", "Socialistische Partij", "Socialistische Partij Anders"],
     ["Onward", "Socialist Party", "Socialist Party Differently"]],

    ["BE", 104,
     ["ECOLO"],
     ["Ecolo"],
     ["Ecolo"]],

    ["BE", 105,
     ["AGALEV", "Groen"],
     ["Agalev", "Groen!"],
     ["Agalev", "Green!"]],

    ["BE", 106,
     ["PRL", "MR"],
     ["Parti Réformateur Libéral", "Mouvement Réformateur"],
     ["Liberal Reformist Party", "Reformist Movement"]],

    ["BE", 107,
     ["VLD", "Open VLD", "PVV"],
     ["Open Vlaamse Liberalen en Democraten",
      "Partij voor Vrijheid en Vooruitgang"],
     ["Open Flemish Liberals and Democrats",
      "Party of Liberty and Progress"]],

    ["BE", 108,
     ["PSC", "CDH", "LE"],
     ["Parti Social Chrétien",
      "Centre Démocrate Humaniste",
      "Les Engagés"],
     ["Christian Social Party",
      "Humanist Democratic Centre",
      "The Engaged"]],

    ["BE", 109,
     ["CVP", "CD&V"],
     ["Christelijke Volkspartij",
      "Christen-Democratisch en Vlaams"],
     ["Christian People’s Party",
      "Christian Democratic & Flemish"]],

    ["BE", 110,
     ["VU", "NVA"],
     ["Volksunie", "Nieuw-Vlaamse Alliantie"],
     ["People’s Union", "New Flemish Alliance"]],

    ["BE", 111,
     ["FDF", "DéFI"],
     ["Front Démocratique des Francophones",
      "Démocrate fédéraliste Independent"],
     ["Francophone Democratic Front",
      "Christian Democratic & Flemish"]],

    ["BE", 112,
     ["VB"],
     ["Vlaams Blok", "Vlaams Belang"],
     ["Flemish Bloc", "Flemish Interest"]],

    ["BE", 113,
     ["MCC"],
     ["Mouvement des Citoyens pour le Changement"],
     ["Citizens’ Movement for Change"]],

    ["BE", 114,
     ["ID21"],
     ["ID21"],
     ["ID21"]],

    ["BE", 115,
     ["FN"],
     ["Front National"],
     ["National Front"]],

    ["BE", 117,
     ["LDD"],
     ["Lijst Dedecker"],
     ["List Dedecker"]],

    ["BE", 118,
     ["SLP", "Spirit"],
     ["Sociaal-Liberale Partij", "Spirit"],
     ["Social-Liberal Party", "Spirit"]],

    ["BE", 119,
     ["PVDA", "PVDA-PTB"],
     ["Partij van de Arbeid van België",
      "Parti du Travail de Belgique"],
     ["Workers’ Party of Belgium"]],

    ["BE", 120,
     ["PP"],
     ["Parti Populaire"],
     ["People’s Party"]],
]

all_party_tables.append(parties_BE)

In [7]:
# danish parties
parties_DK = [
    ["DK", 201,
     ["SD"],
     ["Socialdemokraterne"],
     ["Social Democrats"]],

    ["DK", 202,
     ["RV"],
     ["Det Radikale Venstre"],
     ["Radical Left-Social Liberal Party"]],

    ["DK", 203,
     ["KF"],
     ["Det Konservative Folksparti"],
     ["Conservative People’s Party"]],

    ["DK", 204,
     ["CD"],
     ["Centrum-Demokraterne"],
     ["Centre Democrats"]],

    ["DK", 206,
     ["SF"],
     ["Socialistisk Folkeparti"],
     ["Socialist People’s Party", "Green Left"]],

    ["DK", 210,
     ["KRF"],
     ["Kristeligt Folkeparti"],
     ["Christian People’s Party"]],

    ["DK", 211,
     ["V"],
     ["Venstre, Danmarks Liberale Parti"],
     ["Venstre, Liberal Party of Denmark"]],

    ["DK", 212,
     ["FP"],
     ["Fremskridtspartiet"],
     ["Progress Party"]],

    ["DK", 213,
     ["EL"],
     ["Enhedslisten—De Rød-Grønne"],
     ["Unity List-Red/Green Alliance"]],

    ["DK", 215,
     ["DF"],
     ["Dansk Folkeparti"],
     ["Danish People’s Party"]],

    ["DK", 216,
     ["JuniB"],
     ["JuniBevægelsen"],
     ["June Movement"]],

    ["DK", 217,
     ["FolkB"],
     ["Folkebevægelsen mod EU"],
     ["People’s Movement Against the EU"]],

    ["DK", 218,
     ["LA/NLA"],
     ["Liberal Alliance", "Ny Alliance"],
     ["Liberal Alliance", "New Alliance"]],

    ["DK", 219,
     ["A"],
     ["Alternativet"],
     ["The Alternative"]],

    ["DK", 220,
     ["NB"],
     ["Nye Borgerlige"],
     ["The New Right"]],

    ["DK", 221,
     ["M"],
     ["Moderaterne"],
     ["Moderates"]],

    ["DK", 222,
     ["DD"],
     ["Danmarksdemokraterne"],
     ["Denmark Democrats"]],
]

all_party_tables.append(parties_DK)

In [8]:
# german parties
parties_GE = [
    ["GE", 301,
     ["CDU"],
     ["Christlich Demokratische Union Deutschlands"],
     ["Christian Democratic Union of Germany"]],

    ["GE", 302,
     ["SPD"],
     ["Sozialdemokratische Partei Deutschlands"],
     ["Social Democratic Party of Germany"]],

    ["GE", 303,
     ["FDP"],
     ["Freie Demokratische Partei"],
     ["Free Democratic Party"]],

    ["GE", 304,
     ["Grunen"],
     ["Bündnis ’90; Die Grünen"],
     ["Alliance 90; The Greens"]],

    ["GE", 305,
     ["REP"],
     ["Republikaner"],
     ["Republikaner"]],

    ["GE", 306,
     ["PDS", "Linkspartei", "Linke"],
     ["Partei des Demokratischen Sozialismus",
      "Die Linkspartei",
      "Die Linke"],
     ["Party of Democratic Socialism",
      "Left Party",
      "The Left"]],

    ["GE", 308,
     ["CSU"],
     ["Christlich Soziale Union in Bayern"],
     ["Christian Social Union in Bavaria"]],

    ["GE", 309,
     ["DVU", "NPD"],
     ["Deutsche Volksunion",
      "Nationaldemokratische Partei Deutschlands"],
     ["German People’s Union",
      "National Democratic Party of Germany"]],

    ["GE", 310,
     ["AfD"],
     ["Alternative für Deutschland"],
     ["Alternative for Germany"]],

    ["GE", 311,
     ["Piraten"],
     ["Piratenpartei Deutschland"],
     ["Pirate Party of Germany"]],

    ["GE", 312,
     ["DieTier"],
     ["Mensch Umwelt Tierschutz"],
     ["Human Environment Animal Protection"]],

    ["GE", 313,
     ["BSW"],
     ["Bündnis Sahra Wagenknecht"],
     ["Sahra Wagenknecht Alliance"]],

    ["GE", 314,
     ["FW"],
     ["Freie Wähler"],
     ["Free Voters"]],
]

all_party_tables.append(parties_GE)

In [9]:
# greek parties
parties_GR = [
    ["GR", 401,
     ["PASOK"],
     ["Panellinio Sosialistikó Kínima"],
     ["Panhellenic Socialist Movement"]],

    ["GR", 402,
     ["ND"],
     ["Néa Dimokratía"],
     ["New Democracy"]],

    ["GR", 403,
     ["SYN", "SYRIZA"],
     ["Synaspismós tïs Aristerás kai tïs Proódou",
      "Synaspismó’s Rizospastikís Aristerás"],
     ["Coalition of the Left and Progress",
      "Coalition of the Radical Left"]],

    ["GR", 404,
     ["KKE"],
     ["Kommounistikó Kómma Elládas"],
     ["Communist Party of Greece"]],

    ["GR", 408,
     ["POLA"],
     ["Politiki Anixi"],
     ["Political Spring"]],

    ["GR", 409,
     ["DIKKI"],
     ["Dimokratiko Kinoniko Kinima"],
     ["Democratic Social Movement"]],

    ["GR", 410,
     ["LAOS"],
     ["Laïkós Orthódoxos Synagermós"],
     ["Popular Orthodox Rally"]],

    ["GR", 411,
     ["OP"],
     ["Oikologoi Prasinoi"],
     ["Ecologist Greens"]],

    ["GR", 412,
     ["ANEL"],
     ["Anexartitoi Ellines"],
     ["Independent Greeks"]],

    ["GR", 413,
     ["Potami"],
     ["To Potami"],
     ["The River"]],

    ["GR", 414,
     ["DIMAR"],
     ["Dimokratiki Aristera"],
     ["Democratic Left"]],

    ["GR", 415,
     ["XA"],
     ["Laïkós Sýndesmos—Chrysí Avgí"],
     ["Popular Association—Golden Dawn"]],

    ["GR", 416,
     ["EL"],
     ["Elliniki Lisi"],
     ["Greek Solution"]],

    ["GR", 417,
     ["MR25"],
     ["Métopo Evropaikís Realistikís Anypakoís"],
     ["European Realistic Disobedience Front [MeRa25]"]],

    ["GR", 418,
     ["KIDISO"],
     ["Kinima Dimokraton Sosialiston"],
     ["Movement of Democratic Socialists"]],

    ["GR", 419,
     ["Niki"],
     ["Dimokratikó Patriotikó Kínima – Níki"],
     ["Democratic Patriotic Movement – Victory"]],

    ["GR", 420,
     ["PE"],
     ["Plefsi Eleftherias"],
     ["Course of Freedom"]],

    ["GR", 421,
     ["FL"],
     ["Foní Loyikís"],
     ["Voice of Reason"]],

    ["GR", 422,
     ["Spartans"],
     ["Spartiátes"],
     ["Spartans"]],
]

all_party_tables.append(parties_GR)

In [10]:
# spanish parties
parties_ESP = [
    ["ESP", 501,
     ["PSOE"],
     ["Partido Socialista Obrero Español"],
     ["Spanish Socialist Workers’ Party"]],

    ["ESP", 502,
     ["PP"],
     ["Partido Popular", "Alianza-Partido Popular"],
     ["People’s Party", "People’s Alliance Party"]],

    ["ESP", 504,
     ["IU"],
     ["Izquierda Unida"],
     ["United Left"]],

    ["ESP", 505,
     ["CiU"],
     ["Convergència i Unió"],
     ["Convergence and Unity"]],

    ["ESP", 506,
     ["PNV"],
     ["Euzko Alderdi Jeltzalea/Partido Nacionalista Vasco"],
     ["Basque Nationalist Party"]],

    ["ESP", 507,
     ["EA"],
     ["Eusko Alkartasuna"],
     ["Basque Solidarity"]],

    ["ESP", 508,
     ["EH"],
     ["Euskal Herritarrak"],
     ["We Basque Citizens"]],

    ["ESP", 510,
     ["PAR"],
     ["Partido Aragonés"],
     ["Aragonese Party"]],

    ["ESP", 511,
     ["ERC"],
     ["Esquerra Republicana de Catalunya"],
     ["Republican Left of Catalonia"]],

    ["ESP", 512,
     ["UV"],
     ["Unión Valenciana"],
     ["Valencian Union"]],

    ["ESP", 513,
     ["BNG"],
     ["Bloque Nacionalista Galego"],
     ["Galician Nationalist Bloc"]],

    ["ESP", 514,
     ["VERDE"],
     ["Los Verdes"],
     ["The Greens"]],

    ["ESP", 516,
     ["PA"],
     ["Partido Andalucista"],
     ["Andalusian Party"]],

    ["ESP", 517,
     ["CC"],
     ["Coalición Canaria"],
     ["Coalition Canaria"]],

    ["ESP", 518,
     ["IC"],
     ["Iniciativa per Catalunya"],
     ["Initiative for Catalonia"]],

    ["ESP", 519,
     ["PSC"],
     ["Partit dels Socialistes de Catalunya"],
     ["Catalan Socialist Party"]],

    ["ESP", 520,
     ["CHA"],
     ["Chunta Aragonesista"],
     ["Aragonese Council"]],

    ["ESP", 521,
     ["CpE"],
     ["Coalición por Europa"],
     ["Coalition for Europe"]],

    ["ESP", 522,
     ["EdP-V"],
     ["Europa de los Pueblos-Los Verdes"],
     ["Europe of the Peoples-Greens"]],

    ["ESP", 523,
     ["UPyD"],
     ["Unión Progreso y Democracia"],
     ["Union, Progress, and Democracy"]],

    ["ESP", 524,
     ["EHB"],
     ["Euskal Herria Bildu", "Amaiur"],
     ["Basque Country Unite/Gather", "Amaiur"]],

    ["ESP", 525,
     ["Podemos"],
     ["Podemos", "Unidas Podemos"],
     ["We Can"]],

    ["ESP", 526,
     ["Cs"],
     ["Ciudadanos—Partido de la Ciudadanía"],
     ["Citizens—Party of the Citizenry"]],

    ["ESP", 527,
     ["Vox"],
     ["Vox"],
     ["Voice (Latin)"]],

    ["ESP", 528,
     ["Pais"],
     ["Más Pais"],
     ["More Country"]],

    ["ESP", 550,
     ["PdeCat"],
     ["Partit Demócrata Europeu Catalá (Junts per Catalunya)"],
     ["Catalan European Democratic Party (Together for Catalonia partner)"]],

    ["ESP", 551,
     ["Sumar"],
     ["Sumar"],
     ["Unite"]],

    ["ESP", 552,
     ["Salf"],
     ["Se Acabó La Fiesta"],
     ["The Party is Over"]],

    ["ESP", 553,
     ["UPN"],
     ["Unión del Pueblo Navarro"],
     ["Navarrese People’s Union"]],
]

all_party_tables.append(parties_ESP)

In [11]:
# french parties
parties_FR = [
    ["FR", 601, ["PCF"],
     ["Parti Communiste Français"],
     ["French Communist Party"]],

    ["FR", 602, ["PS"],
     ["Parti Socialiste"],
     ["Socialist Party"]],

    ["FR", 603, ["PRG"],
     ["Parti Radical de Gauche"],
     ["Left Radical Party"]],

    ["FR", 605, ["VERTS", "EELV"],
     ["Les Verts", "Europe Écologie Les Verts"],
     ["Green Party", "Europe Ecology—The Greens"]],

    ["FR", 609, ["RPR", "UMP", "LR"],
     ["Rassemblement pour la République",
      "Union pour un Mouvement Populaire",
      "Les Républicains"],
     ["Rally for the Republic",
      "Union for Popular Movement",
      "The Republicans"]],

    ["FR", 610, ["FN", "RN"],
     ["Front National", "Rassemblement national"],
     ["National Front", "National Rally"]],

    ["FR", 612, ["RPF/MPF", "MPF"],
     ["Rassemblement pour la France/Movement Pour la France",
      "Mouvement Pour la France"],
     ["Rally for France/Movement for France",
      "Movement for France"]],

    ["FR", 613, ["UDF", "MoDem"],
     ["Union pour la Démocratie Française",
      "Mouvement Démocrate"],
     ["Union for French Democracy",
      "Democratic Movement"]],

    ["FR", 614, ["LO-LCR"],
     ["Lutte Ouvrière/Ligue communiste révolutionnaire"],
     ["Workers’ Struggle/Revolutionary Communist League"]],

    ["FR", 615, ["DL"],
     ["Démocratie Libérale"],
     ["Liberal Democracy"]],

    ["FR", 617, ["MEI"],
     ["Mouvement Écologiste Indépendant"],
     ["Independent Ecological Movement"]],

    ["FR", 618, ["D"],
     ["La Droite"],
     ["The Right"]],

    ["FR", 619, ["CPNT"],
     ["Chasse, Pêche, Nature, Traditions"],
     ["Hunting, Fishing, Nature, Tradition Party"]],

    ["FR", 620, ["MN"],
     ["Mouvement National Républicain"],
     ["National Republican Movement"]],

    ["FR", 621, ["NC"],
     ["Nouveau Centre"],
     ["New Center"]],

    ["FR", 622, ["PRV"],
     ["Parti radical"],
     ["Radical Party"]],

    ["FR", 623, ["AC"],
     ["Alliance centriste"],
     ["Centrist Alliance"]],

    ["FR", 624, ["PG"],
     ["Parti de Gauche"],
     ["Left Party"]],

    ["FR", 625, ["Ens"],
     ["Ensemble"],
     ["Together"]],

    ["FR", 626, ["RE", "LREM"],
     ["Renaissance", "La République en Marche!"],
     ["Renaissance", "The Republic Onwards!"]],

    ["FR", 627, ["FI"],
     ["La France Insoumise"],
     ["Unbowed France"]],

    ["FR", 628, ["DLF"],
     ["Debout la France"],
     ["France Arise"]],

    ["FR", 630, ["REC"],
     ["Reconquête"],
     ["Reconquest"]],

    ["FR", 631, ["Horizons"],
     ["Horizons"],
     ["Horizons"]],
]

all_party_tables.append(parties_FR)

In [12]:
# irish parties
parties_IRL = [
    ["IRL", 701, ["FF"],
     ["Fianna Fáil"],
     ["Soldiers of Destiny"]],

    ["IRL", 702, ["FG"],
     ["Fine Gael"],
     ["Family of the Irish"]],

    ["IRL", 703, ["Lab"],
     ["Páirtí Lucht Oibre"],
     ["Labour"]],

    ["IRL", 705, ["GP"],
     ["Comhaontas Glas"],
     ["Green Party"]],

    ["IRL", 707, ["SF"],
     ["Sinn Féin"],
     ["We Ourselves"]],

    ["IRL", 708, ["SP"],
     ["Páirtí Sóisialach"],
     ["Socialist Party"]],

    ["IRL", 709, ["PBPA", "S-PBP"],
     ["People Before Profit Alliance",
      "Dlúthpháirtíocht–Pobal Roimh Bhrabús"],
     ["People Before Profit Alliance",
      "Solidarity—People Before Profit"]],

    ["IRL", 710, ["DS"],
     ["Daonlathach Sóisialta"],
     ["Social Democrats"]],

    ["IRL", 711, ["I4C"],
     ["Independents for Change"],
     ["Independents for Change"]],

    ["IRL", 712, ["RI"],
     ["Renua Ireland"],
     ["Renua Ireland"]],

    ["IRL", 713, ["Aon"],
     ["Aontú"],
     ["Unity"]],
]

all_party_tables.append(parties_IRL)

In [13]:
# italian parties
parties_IT = [
    ["IT", 802, ["PDS", "DS"],
     ["Partito Democratico della Sinistra",
      "Democratici di Sinistra"],
     ["Democratic Party of the Left",
      "Democrats of the Left"]],

    ["IT", 803, ["RC"],
     ["Partito della Rifondazione Comunista"],
     ["Communist Refoundation Party"]],

    ["IT", 805, ["AN"],
     ["Alleanza Nazionale"],
     ["National Alliance"]],

    ["IT", 806, ["PRI"],
     ["Partito Repubblicano Italiano"],
     ["Italian Republican Party"]],

    ["IT", 807, ["PSDI", "SDI"],
     ["Partito Socialista Democratico Italiano",
      "Socialisti Democratici Italiani"],
     ["Italian Democratic Socialist Party",
      "Italian Democratic Socialists"]],

    ["IT", 808, ["FdV", "Verdi"],
     ["Federazione dei Verdi"],
     ["Federation of Greens"]],

    ["IT", 811, ["LEGA", "LN"],
     ["Lega",
      "Lega Nord"],
     ["League",
      "Northern League"]],

    ["IT", 813, ["RAD"],
     ["Radicali Italiani"],
     ["Italian Radicals"]],

    ["IT", 814, ["CCD", "UDC"],
     ["Centro Cristiano Democratico",
      "Unione dei Democratici Cristiani e dei Democratici di Centro",
      "Unione di Centro"],
     ["Christian Democratic Center",
      "Union of Christian and Center Democrats",
      "Union of the Centre"]],

    ["IT", 815, ["FI"],
     ["Forza Italia–Il Popolo della Libertà"],
     ["Forward Italy–The People of Freedom"]],

    ["IT", 817, ["CU/UD"],
     ["Unione Democratica"],
     ["Democratic Union"]],

    ["IT", 818, ["CDU"],
     ["Cristiani Democratici Uniti"],
     ["United Christian Democrats"]],

    ["IT", 819, ["DEM", "DL"],
     ["I Democratici",
      "Democrazia è Libertà—La Margherita"],
     ["The Democrats",
      "Democracy is Freedom—The Daisy"]],

    ["IT", 820, ["LB"],
     ["Lista Bonino"],
     ["Bonino List"]],

    ["IT", 821, ["MS"],
     ["Movimento Sociale Fiamma Tricolore"],
     ["Tricolor Flame Social Movement"]],

    ["IT", 822, ["PdUP"],
     ["Partito di Unità Proletaria"],
     ["Party of Proletarian Unity for Communism"]],

    ["IT", 823, ["PPI"],
     ["Partito Popolare Italiano"],
     ["Italian Popular Party"]],

    ["IT", 824, ["PsDA"],
     ["Partito Sardo d’Azione"],
     ["Sardinian Action Party"]],

    ["IT", 825, ["RI"],
     ["Rinnovamento Italiano"],
     ["Italian Renewal–Dini List"]],

    ["IT", 826, ["SEG"],
     ["Patto Segni"],
     ["Segni Pact"]],

    ["IT", 827, ["SVP"],
     ["Südtiroler Volkspartei"],
     ["South Tyrolean People’s Party"]],

    ["IT", 828, ["IdV"],
     ["Di Pietro–Italia dei Valori"],
     ["Italy of Values"]],

    ["IT", 829, ["UDEUR"],
     ["Popolari–UDEUR"],
     ["Popular–UDEUR"]],

    ["IT", 830, ["PP"],
     ["Partito dei Pensionati"],
     ["Pensioners’ Party"]],

    ["IT", 832, ["+E", "MRE"],
     ["Più Europa",
      "Movimento Repubblicani Europei"],
     ["More Europe",
      "European Republican Movement"]],

    ["IT", 835, ["NPSI"],
     ["Nuovo Partito Socialista Italiano"],
     ["New Italian Socialist Party"]],

    ["IT", 836, ["PdCI"],
     ["Partito dei Comunisti Italiani"],
     ["Party of Italian Communists"]],

    ["IT", 837, ["PD"],
     ["Partito Democratico"],
     ["Democratic Party"]],

    ["IT", 838, ["AVS", "SL", "SEL"],
     ["Alleanza Verdi e Sinistra",
      "Sinistra e Libertà",
      "Sinistra Ecologia Libertà",
      "Sinistra Italiana"],
     ["Green & Left Alliance",
      "Left and Freedom",
      "Left Ecology Freedom",
      "Italian Left—Left Ecology Freedom"]],

    ["IT", 839, ["PSI"],
     ["Partito Socialista Italiano"],
     ["Italian Socialist Party"]],

    ["IT", 840, ["MpA"],
     ["Movimento per le Autonomie"],
     ["Movement for Autonomies"]],

    ["IT", 842, ["SD"],
     ["Sinistra Democratica"],
     ["Democratic Left"]],

    ["IT", 843, ["CD"],
     ["Centro Democratico—Diritti e Libertà"],
     ["Democratic Centre"]],

    ["IT", 844, ["FDL"],
     ["Fratelli d’Italia"],
     ["Brothers of Italy"]],

    ["IT", 845, ["M5S"],
     ["MoVimento Cinque Stelle"],
     ["Five Star Movement"]],

    ["IT", 846, ["SC"],
     ["Scelta Civica"],
     ["Civic Choice"]],

    ["IT", 847, ["VdA"],
     ["Vallée d’Aoste"],
     ["Aosta Valley"]],

    ["IT", 848, ["NCD"],
     ["Nuovo Centrodestra"],
     ["New Centre-Right"]],

    ["IT", 849, ["A-IV"],
     ["Azione–Italia Viva"],
     ["Action–Italy Alive"]],

    ["IT", 850, ["EV"],
     ["Europa Verde"],
     ["Green Europe"]],
]

all_party_tables.append(parties_IT)

In [14]:
# parties of netherlands
parties_NL = [
    ["NL", 1001, ["CDA"],
     ["Christen-Democratisch Appèl"],
     ["Christian Democratic Appeal"]],

    ["NL", 1002, ["PvdA"],
     ["Partij van de Arbeid"],
     ["Labour Party"]],

    ["NL", 1003, ["VVD"],
     ["Volkspartij voor Vrijheid en Democratie"],
     ["People’s Party for Freedom and Democracy"]],

    ["NL", 1004, ["D66"],
     ["Democraten 66"],
     ["Democrats 66"]],

    ["NL", 1005, ["GL"],
     ["GroenLinks"],
     ["GreenLeft"]],

    ["NL", 1006, ["SGP"],
     ["Staatkundig Gereformeerde Partij"],
     ["Reformed Political Party"]],

    ["NL", 1007, ["GPV"],
     ["Gereformeerd Politiek Verbond"],
     ["Reformed Political League"]],

    ["NL", 1008, ["RPF"],
     ["Reformatorische Politieke Federatie"],
     ["Reformational Political Federation"]],

    ["NL", 1009, ["CD"],
     ["Centrum Democraten"],
     ["Center Democrats"]],

    ["NL", 1014, ["SP"],
     ["Socialistische Partij"],
     ["Socialist Party"]],

    ["NL", 1015, ["LPF"],
     ["Lijst Pim Fortuyn"],
     ["Lijst Pim Fortuyn"]],

    ["NL", 1016, ["CU"],
     ["ChristenUnie"],
     ["ChristianUnion"]],

    ["NL", 1017, ["PVV"],
     ["Partij voor de Vrijheid"],
     ["Party for Freedom"]],

    ["NL", 1018, ["PvdD"],
     ["Partij voor de Dieren"],
     ["Party for the Animals"]],

    ["NL", 1020, ["50PLUS"],
     ["50PLUS"],
     ["50PLUS"]],

    ["NL", 1050, ["DENK"],
     ["Beweging DENK"],
     ["Think"]],

    ["NL", 1051, ["FvD"],
     ["Forum voor Democratie"],
     ["Forum for Democracy"]],

    ["NL", 1052, ["Volt"],
     ["Volt Nederland"],
     ["Volt Netherlands"]],

    ["NL", 1054, ["BBB"],
     ["BoerBurgerBeweging"],
     ["Farmer-Citizen Movement"]],

    ["NL", 1056, ["NSC"],
     ["Nieuw Sociaal Contract"],
     ["New Social Contract"]],
]

all_party_tables.append(parties_NL)

In [15]:
# british parties
parties_UK = [
    ["UK", 1101, ["Cons"],
     ["Conservative Party"],
     ["Conservative Party"]],

    ["UK", 1102, ["Lab"],
     ["Labour Party"],
     ["Labour Party"]],

    ["UK", 1104, ["LibDem"],
     ["Liberal Democratic Party"],
     ["Liberal Democratic Party"]],

    ["UK", 1105, ["SNP"],
     ["Scottish National Party"],
     ["Scottish National Party"]],

    ["UK", 1106, ["Plaid"],
     ["Plaid Cymru"],
     ["Party of Wales"]],

    ["UK", 1107, ["Green"],
     ["Green Party"],
     ["Green Party"]],

    ["UK", 1108, ["UKIP"],
     ["United Kingdom Independence Party"],
     ["United Kingdom Independence Party"]],

    ["UK", 1109, ["BNP"],
     ["British National Party"],
     ["British National Party"]],

    ["UK", 1110, ["Brexit/Reform"],
     ["Brexit Party/Reform UK"],
     ["Brexit Party/Reform UK"]],
]

all_party_tables.append(parties_UK)

In [16]:
# portugese parties
parties_POR = [
    ["POR", 1201, ["CDU"],
     ["Coligação Democrática Unitária"],
     ["Democratic Unitarian Coalition"]],

    ["POR", 1202, ["CDS", "PP", "CDS-PP"],
     ["Centro Democrático e Social",
      "Partido Popular",
      "Centro Democrático e Social—Partido Popular"],
     ["Democratic and Social Center",
      "People’s Party",
      "CDS—People’s Party"]],

    ["POR", 1205, ["PS"],
     ["Partido Socialista"],
     ["Socialist Party"]],

    ["POR", 1206, ["PPD", "PSD"],
     ["Partido Popular Democrático",
      "Partido Social Democrata"],
     ["Democratic People’s Party",
      "Social Democratic Party"]],

    ["POR", 1208, ["BE"],
     ["Bloco de Esquerda"],
     ["Left Bloc"]],

    ["POR", 1209, ["MPT"],
     ["Partido da Terra"],
     ["Earth Party"]],

    ["POR", 1210, ["PCP"],
     ["Partido Comunista Português"],
     ["Portuguese Communist Party"]],

    ["POR", 1211, ["PEV"],
     ["Partido Ecologista “Os Verdes”"],
     ["Ecologist Party “The Greens”"]],

    ["POR", 1250, ["PAN"],
     ["Pessoas—Animais—Natureza"],
     ["People—Animals—Nature"]],

    ["POR", 1251, ["L"],
     ["Livre"],
     ["Free"]],

    ["POR", 1252, ["IL"],
     ["Iniciativa Liberal"],
     ["Liberal Initiative"]],

    ["POR", 1253, ["CHEGA"],
     ["Chega"],
     ["Enough"]],
]

all_party_tables.append(parties_POR)

In [17]:
# parties of austria
parties_AUS = [
    ["AUS", 1301, ["SPÖ"],
     ["Sozialdemokratische Partei Österreichs"],
     ["Social Democratic Party of Austria"]],

    ["AUS", 1302, ["ÖVP"],
     ["Österreichische Volkspartei"],
     ["Austrian People’s Party"]],

    ["AUS", 1303, ["FPÖ"],
     ["Freiheitliche Partei Österreichs"],
     ["Freedom Party of Austria"]],

    ["AUS", 1304, ["Grüne"],
     ["Die Grünen"],
     ["The Austrian Green Party"]],

    ["AUS", 1306, ["LIF", "NEOS"],
     ["Liberales Forum",
      "NEOS—Das Neue Österreich und Liberales Forum"],
     ["Liberal Forum",
      "NEOS—The New Austria and Liberal Forum"]],

    ["AUS", 1307, ["BZÖ"],
     ["Bündnis Zukunft Österreich"],
     ["Alliance for the Future of Austria"]],

    ["AUS", 1308, ["MARTIN"],
     ["Liste Dr. Hans-Peter Martin"],
     ["Hans-Peter Martin’s List"]],

    ["AUS", 1310, ["TeamStronach"],
     ["Team Stronach für Österreich"],
     ["Team Stronach for Austria"]],
]

all_party_tables.append(parties_AUS)

In [18]:
# finnish parties
parties_FIN = [
    ["FIN", 1401, ["SDP"],
     ["Suomen Sosialidemokraattinen Puolue"],
     ["Social Democratic Party of Finland"]],

    ["FIN", 1402, ["KOK"],
     ["Kansallinen Kokoomus"],
     ["National Coalition Party"]],

    ["FIN", 1403, ["KESK"],
     ["Suomen Keskusta"],
     ["Finnish Center Party"]],

    ["FIN", 1404, ["VAS"],
     ["Vasemmistoliitto",
      "Demokraattinen Liitto—Demokratiska Förbundet"],
     ["Left Alliance",
      "Democratic Union"]],

    ["FIN", 1405, ["PS"],
     ["Perussuomalaiset"],
     ["True Finns",
      "The Finns party"]],

    ["FIN", 1406, ["RKP/SFP"],
     ["Ruotsalainen kansanpuolue/Svenska folkpartiet",
      "Suomen ruotsalainen kansanpuolue/Svenska folkpartiet i Finland"],
     ["Swedish People’s Party",
      "The Swedish People’s Party of Finland"]],

    ["FIN", 1408, ["VIHR"],
     ["Vihreä Liitto"],
     ["Green League"]],

    ["FIN", 1409, ["SKL", "KD"],
     ["Suomen Kristillinen Liitto",
      "Kristillisdemokraatit"],
     ["Finnish Christian League",
      "Christian Democrats"]],

    ["FIN", 1411, ["EKA"],
     ["Eläkeläiset Kansan Asialla"],
     ["Pensioners for the People"]],

    ["FIN", 1412, ["REM"],
     ["Remonttiryhmä"],
     ["Reform Group"]],

    ["FIN", 1413, ["KIPU"],
     ["Kirjava “puolue”—Elonkehän puolesta"],
     ["Ecological Party"]],

    ["FIN", 1414, ["AS"],
     ["Åländsk Samling (ÅS)"],
     ["For Åland (Åland Coalition)"]],
]

all_party_tables.append(parties_FIN)

In [19]:
# swedish parties
parties_SV = [
    ["SV", 1601, ["V"],
     ["Vänsterpartiet"],
     ["Left Party"]],

    ["SV", 1602, ["SAP"],
     ["Arbetarpartiet–Socialdemokraterna",
      "Sveriges socialdemokratiska arbetareparti"],
     ["Worker’s Party–Social Democrats",
      "Swedish Social Democratic Party"]],

    ["SV", 1603, ["C"],
     ["Centerpartiet"],
     ["Center Party"]],

    ["SV", 1604, ["FP", "L"],
     ["Folkpartiet liberalerna",
      "Liberalerna"],
     ["Liberal People’s Party"]],

    ["SV", 1605, ["M"],
     ["Moderaterna",
      "Moderata Samlingspartiet"],
     ["Moderate Party"]],

    ["SV", 1606, ["KD"],
     ["Kristdemokraterna"],
     ["Christian-Democrats"]],

    ["SV", 1607, ["MP"],
     ["Miljöpartiet de Gröna"],
     ["Environment Party—The Greens"]],

    ["SV", 1608, ["NyD"],
     ["Ny Demokrati"],
     ["New Democracy"]],

    ["SV", 1609, ["JL"],
     ["Junilistan"],
     ["June List"]],

    ["SV", 1610, ["SD"],
     ["Sverigedemokraterna"],
     ["Sweden Democrats"]],

    ["SV", 1611, ["PIRAT"],
     ["Piratpartiet"],
     ["Pirate Party"]],

    ["SV", 1612, ["FI"],
     ["Feministiskt initiativ"],
     ["Feminist Initiative"]],
]

all_party_tables.append(parties_SV)

In [20]:
# bulgarian parties
parties_BUL = [
    ["BUL", 2001, ["NDSV"],
     ["Nacionalno Dviženie Simeon Vtori"],
     ["National Movement Simeon II"]],

    ["BUL", 2002, ["ODS", "SDS"],
     ["Obedineni Demokratichni Sili",
      "Sayuz na demokratichnite sili"],
     ["United Democratic Forces",
      "Union of Democratic Forces"]],

    ["BUL", 2003, ["KzB", "BSP"],
     ["Koalicija za Bălgariya",
      "BSP—Lyava Bălgariya",
      "Bălgarska sotsialisticheska partiya"],
     ["Coalition for Bulgaria",
      "BSP—Left Bulgaria",
      "Bulgarian Socialist Party"]],

    ["BUL", 2004, ["DPS", "APS"],
     ["Dvizenie za Prava i Svobodi",
      "Alianzs za prava i svobodi"],
     ["Movement for Rights and Freedoms",
      "Alliance for Rights and Freedoms"]],

    ["BUL", 2005, ["G-VMRO/VMRO-BND"], # known as VMRO-BND (NOTE)
     ["Dvizenie Gergiovden",
      "VMRO—Bălgarsko Natsionalno Dvizhenie"],
     ["Gergiovden-VMRO",
      "VMRO—Bulgarian National Movement"]],

    ["BUL", 2006, ["KSII"],
     ["Koalicija Simeon II"],
     ["Coalition Simeon II"]],

    ["BUL", 2007, ["NOA", "Ataka"],
     ["Nacionalno Obedinenie Ataka",
      "Ataka"],
     ["National Union Attack",
      "Attack"]],

    ["BUL", 2008, ["DSB"],
     ["Demokrati za Silna Bălgariya"],
     ["Democrats for a Strong Bulgaria"]],

    ["BUL", 2009, ["BNS"],
     ["Bălgarski Naroden Săyuz"],
     ["Bulgarian People’s Union"]],

    ["BUL", 2010, ["GERB"],
     ["Grazhdani za evropeysko razvitie na Bălgariya"],
     ["Citizens for European Development of Bulgaria"]],

    ["BUL", 2011, ["L"],
     ["Lider"],
     ["Lider"]],

    ["BUL", 2012, ["RZS"],
     ["Red, zakonnost i spravedlivost"],
     ["Order, Lawfulness, Justice"]],

    ["BUL", 2013, ["DBG"],
     ["Dvizenie Bulgaria na grazhdanite"],
     ["Bulgaria for Citizens Movement"]],

    ["BUL", 2014, ["NFSB"],
     ["Natsionalen Front za Spasenie na Bulgaria"],
     ["National Front for the Salvation of Bulgaria"]],

    ["BUL", 2015, ["BBT"],
     ["Bulgaria bez Tsenzura"],
     ["Bulgaria without Censorship"]],

    ["BUL", 2016, ["ABV"],
     ["Alternativa za Bălgarsko Vazrazhdane"],
     ["Alternative for Bulgarian Revival"]],

    ["BUL", 2017, ["Volya"],
     ["Volya"],
     ["Will"]],

    ["BUL", 2018, ["DB"],
     ["Da Bulgaria"],
     ["Yes, Bulgaria!"]],

    ["BUL", 2019, ["ST", "ITN"],
     ["Slavi Trifonov",
      "Ima takav narod"],
     ["Party of Slavi Trifonov",
      "There is Such a People"]],

    ["BUL", 2020, ["Vaz"],
     ["Vazrazhdane"],
     ["Revival"]],

    ["BUL", 2021, ["PP"],
     ["Prodalzhavame promyanata"],
     ["We Continue the Change"]],

    ["BUL", 2022, ["DPS-Novo"],
     ["DPS – Novo nachalo"],
     ["DPS – A New Beginning"]],

    ["BUL", 2023, ["MECh"],
     ["Moral, Edinstvo, Chest"],
     ["Morality, Unity, Honour"]],

    ["BUL", 2024, ["Vel"],
     ["Velichie"],
     ["Greatness"]],
]

all_party_tables.append(parties_BUL)

In [21]:
# czech parties
parties_CZECH = [
    ["CZECH", 2101, ["CSSD"],
     ["Česká strana sociálně demokratická"],
     ["Czech Social Democratic Party"]],

    ["CZECH", 2102, ["ODS"],
     ["Občanská demokratická strana"],
     ["Civic Democratic Party"]],

    ["CZECH", 2103, ["KSCM"],
     ["Komunistická strana Čech a Moravy"],
     ["Communist Party of Bohemia and Moravia"]],

    ["CZECH", 2104, ["KDU-CSL"],
     ["Křesťanská demokratická unie—Československá strana lidová"],
     ["Christian Democratic Union—Czechoslovak People’s Party"]],

    ["CZECH", 2105, ["US-DEU"],
     ["Unie Svobody—Demokratická unie"],
     ["Freedom Union—Democratic Union"]],

    ["CZECH", 2106, ["SNK-ED"],
     ["Sdružení nezávislých kandidátů—Evropští demokraté"],
     ["Union of Independents—European Democrats"]],

    ["CZECH", 2107, ["SZ"],
     ["Strana zelených"],
     ["Green Party"]],

    ["CZECH", 2108, ["Nezavisli"],
     ["Nezávislí"],
     ["Independents"]],

    ["CZECH", 2109, ["TOP09"],
     ["Top09",
      "Tradice Odpovědnost Prosperita 09"],
     ["Top09",
      "Tradition Responsibility Prosperity 09"]],

    ["CZECH", 2110, ["VV"],
     ["Věci Veřejné"],
     ["Public Affairs"]],

    ["CZECH", 2111, ["ANO2011"],
     ["ANO 2011",
      "Akce nespokojených občanů"],
     ["ANO 2011",
      "Action of Dissatisfied Citizens"]],

    ["CZECH", 2112, ["USVIT"],
     ["Úsvit přímé demokracie"],
     ["Dawn of Direct Democracy"]],

    ["CZECH", 2113, ["SVOBODNI"],
     ["Strana svobodných občanů"],
     ["Party of Free Citizens"]],

    ["CZECH", 2114, ["Pirates"],
     ["Česká pirátská strana"],
     ["Czech Pirate Party"]],

    ["CZECH", 2115, ["SPD"],
     ["Svoboda a přímá demokracie Tomio Okamura"],
     ["Freedom and Direct Democracy Tomio Okamura"]],

    ["CZECH", 2116, ["STAN"],
     ["Starostové a nezávislí"],
     ["Mayors and Independents"]],

    ["CZECH", 2117, ["Přísaha"],
     ["Přísaha"],
     ["Oath"]],

    ["CZECH", 2118, ["MOTO"],
     ["Motoristé sobě"],
     ["Motorists for Themselves"]],
]

all_party_tables.append(parties_CZECH)

In [22]:
# parties of Estonia
parties_EST = [
    ["EST", 2201, ["IRL", "Isamaa"],
     ["Erakond Isamaa ja Res Publica Liit",
      "Isamaa"],
     ["Pro Patria & Res Publica Union",
      "Fatherland"]],

    ["EST", 2202, ["EK"],
     ["Eesti Keskerakond"],
     ["Estonian Center Party"]],

    ["EST", 2203, ["ER"],
     ["Eesti Reformierakond"],
     ["Estonian Reform Party"]],

    ["EST", 2204, ["SDE"],
     ["Sotsiaaldemokraatlik Erakond"],
     ["Social Democratic Party"]],

    ["EST", 2205, ["KP"],
     ["Eestimaa ühendatud Rahvapartei"],
     ["Constitution Party"]],

    ["EST", 2206, ["ERL"],
     ["Eestimaa Rahvaliit"],
     ["Estonian People’s Union"]],

    ["EST", 2207, ["EER"],
     ["Erakond Eestimaa Rohelised"],
     ["Estonian Greens"]],

    ["EST", 2208, ["EVE"],
     ["Eesti Vabaerakond"],
     ["Estonian Free Party"]],

    ["EST", 2209, ["EKRE"],
     ["Eesti Konservatiivne Rahvaerakond"],
     ["Conservative People’s Party"]],

    ["EST", 2210, ["E200"],
     ["Erakond Eesti 200"],
     ["Estonia 200"]],

    ["EST", 2211, ["Parempoolsed"],
     ["Parempoolsed"],
     ["The Right"]],

    ["EST", 2212, ["Vasakliit"],
     ["Vasakliit"],
     ["Left Alliance"]],
]

all_party_tables.append(parties_EST)

In [23]:
# hungarian parties
parties_HUNG = [
    ["HUNG", 2301, ["MSzP"],
     ["Magyar Szocialista Párt"],
     ["Hungarian Socialist Party"]],

    ["HUNG", 2302, ["Fidesz-M", "Fidesz-KDNP"],
     ["Fidesz—Magyar Polgári Szövetség",
      "Fidesz—Kereszténydemokrata Néppárt"],
     ["Fidesz—Hungarian Civic Union",
      "Fidesz—Christian Democratic People’s Party"]],

    ["HUNG", 2303, ["MDF"],
     ["Magyar Demokrata Fórum"],
     ["Hungarian Democratic Forum"]],

    ["HUNG", 2304, ["SzDSz"],
     ["Szabad Demokraták Szövetsége"],
     ["Alliance of Free Democrats"]],

    ["HUNG", 2305, ["MIEP"],
     ["Magyar Igazság és Élet Pártja"],
     ["Hungarian Justice and Life Party"]],

    ["HUNG", 2306, ["CP"],
     ["Centrum Párt"],
     ["Centre Party"]],

    ["HUNG", 2307, ["KDNP"],
     ["Kereszténydemokrata Néppárt"],
     ["Christian Democratic People’s Party"]],

    ["HUNG", 2308, ["JOBBIK"],
     ["Jobbik Magyarországért Mozgalom"],
     ["Jobbik—Movement for a Better Hungary"]],

    ["HUNG", 2309, ["LMP-MZP"],
     ["Lehet Más a Politika—Magyarország Zöld Pártja"],
     ["Politics can be Different—Hungary’s Green Party"]],

    ["HUNG", 2310, ["E14"],
     ["Együtt 2014",
      "Együtt 2019"],
     ["Together 2014",
      "Together 2019"]],

    ["HUNG", 2311, ["DK"],
     ["Demokratikus Koalíció"],
     ["Democratic Coalition"]],

    ["HUNG", 2314, ["MM"],
     ["Momentum Mozgalom"],
     ["Momentum Movement"]],

    ["HUNG", 2315, ["MKKP"],
     ["Magyar Kétfarkú Kutya Párt"],
     ["Hungarian Two Tailed Dog Party"]],

    ["HUNG", 2316, ["MHM"],
     ["Mi Hazánk Mozgalom"],
     ["Our Homeland Movement"]],

    ["HUNG", 2318, ["Tisza"],
     ["Tisztelet és Szabadság Pártja"],
     ["Tisza—Respect and Freedom Party"]],
]

all_party_tables.append(parties_HUNG)

In [24]:
# latvian parties
parties_LAT = [
    ["LAT", 2401, ["JL"],
     ["Jaunais Laiks"],
     ["New Era Party"]],

    ["LAT", 2402, ["PCTVL", "LKS"],
     ["Par Cilvēka Tiesībām Vienotā Latvijā",
      "Latvijas Krievu savienība"],
     ["For Human Rights in United Latvia",
      "Latvian Russian Union"]],

    ["LAT", 2403, ["TP"],
     ["Tautas Partija"],
     ["People’s Party"]],

    ["LAT", 2404, ["LPP"],
     ["Latvijas Pirmā Partija"],
     ["Latvia’s First Party"]],

    ["LAT", 2405, ["ZZS"],
     ["Zaļo un Zemnieku Savienība"],
     ["Union of Greens and Farmers"]],

    ["LAT", 2406, ["TB-LNNK", "NA"],
     ["Nacionālā apvienība “Visu Latvijai!”—Tēvzemei un Brīvībai/LNNK"],
     ["National Alliance “All For Latvia!”—For Fatherland and Freedom/LNNK"]],

    ["LAT", 2407, ["LC"],
     ["Latvijas Ceļš"],
     ["Latvia’s Way"]],

    ["LAT", 2408, ["LSDSP"],
     ["Latvijas Sociāldemokrātiskā Strādnieku Partija"],
     ["Latvian Social Democratic Labour Party"]],

    ["LAT", 2410, ["SC", "SDPS"],
     ["Saskaņas Centrs",
      "Sociāldemokrātiskā Partija “Saskaņa”"],
     ["Harmony Centre",
      "Social Democratic Party “Harmony”"]],

    ["LAT", 2411, ["ZRP"],
     ["Zatlera Reformu Partija"],
     ["Zatler’s Reform Party"]],

    ["LAT", 2412, ["V", "Vienotiba"],
     ["Vienotība",
      "Jaunā Vienotība"],
     ["Unity",
      "New Unity"]],

    ["LAT", 2413, ["NSL"],
     ["No sirds Latvijai"],
     ["For Latvia from the Heart"]],

    ["LAT", 2414, ["LRA"],
     ["Latvijas Reģionu apvienība"],
     ["Latvian Association of Regions"]],

    ["LAT", 2415, ["KPV LV"],
     ["Kam pieder valsts?"],
     ["Who owns the state?"]],

    ["LAT", 2416, ["JKP"],
     ["Jaunā konservatīvā partija"],
     ["New Conservative Party"]],

    ["LAT", 2417, ["AP!"],
     ["Attīstībai/Par!"],
     ["Development/For!"]],

    ["LAT", 2418, ["S!"],
     ["Stabilitātei!"],
     ["For Stability!"]],

    ["LAT", 2419, ["LPV"],
     ["Latvija pirmajā vietā"],
     ["Latvia First"]],

    ["LAT", 2420, ["Pro"],
     ["Progresīvie"],
     ["The Progressives"]],
]

all_party_tables.append(parties_LAT)

In [25]:
# lithuanian parties
parties_LITH = [
    ["LITH", 2501, ["LSDP"],
     ["Lietuvos socialdemokratų partija"],
     ["Social Democratic Party of Lithuania"]],

    ["LITH", 2502, ["NDP"],
     ["Naujosios demokratijos partija"],
     ["New Democratic Party"]],

    ["LITH", 2503, ["LRS"],
     ["Lietuvos rusų sąjunga"],
     ["Lithuanian Russian Union"]],

    ["LITH", 2504, ["NS"],
     ["Naujoji sąjunga (socialliberalai)"],
     ["New Union (Social Liberals)"]],

    ["LITH", 2505, ["LLS", "LiCS"],
     ["Lietuvos liberalų sąjunga",
      "Liberalių ir centro sąjunga"],
     ["Liberal Union of Lithuania",
      "Liberal and Centre Union"]],

    ["LITH", 2506, ["TS-LKD"],
     ["Tėvynės sąjunga—Lietuvos krikščionys demokratai",
      "Tėvynės sąjunga"],
     ["Homeland Union—Lithuanian Christian Democrats",
      "Homeland Union"]],

    ["LITH", 2507, ["LVP", "LVLS", "LVZS"],
     ["Lietuvos valstiečių partija",
      "Valstiečių ir Naujosios demokratijos partijų sąjunga",
      "Lietuvos valstiečių ir Žaliųjų sąjunga",
      "Lietuvos valstiečių sąjunga"],
     ["Lithuanian Peasant Party",
      "Lithuanian Peasant Popular Union",
      "Lithuanian Peasant and Greens Union",
      "Lithuanian Peasant Union"]],

    ["LITH", 2508, ["LKDP"],
     ["Lietuvos krikščionys demokratai"],
     ["Lithuanian Christian Democrats"]],

    ["LITH", 2509, ["LCS"],
     ["Lietuvos centro sąjunga"],
     ["Center Union of Lithuania"]],

    ["LITH", 2510, ["NKS"],
     ["Nuosaikiųjų konservatorių sąjunga"],
     ["Moderate Conservative Union"]],

    ["LITH", 2511, ["LLRA"],
     ["Lietuvos lenkų rinkimų akcija"],
     ["Electoral Action of Lithuania’s Poles"]],

    ["LITH", 2512, ["LLS"],
     ["Lietuvos laisvės sąjunga"],
     ["Lithuanian Liberty Union"]],

    ["LITH", 2513, ["JL-PKS"],
     ["Lietuvių Nacionalinė Partija – “Jaunoji Lietuva”"],
     ["Young Lithuania—New Nationalists and Political Prisoner’s Union"]],

    ["LITH", 2514, ["MKDS"],
     ["Moderniųjų krikščionių demokratų sąjunga"],
     ["Modern Christian-Democratic Union"]],

    ["LITH", 2515, ["TT"],
     ["Tvarka ir teisingumas"],
     ["Order and Justice"]],

    ["LITH", 2516, ["DP"],
     ["Darbo partija"],
     ["Labour Party"]],

    ["LITH", 2517, ["TPP"],
     ["Tautos prisikėlimo partija"],
     ["National Resurrection Party"]],

    ["LITH", 2518, ["LRLS/LS"],
     ["Lietuvos Respublikos Liberalų Sąjūdis",
      "Liberalų Sąjūdis"],
     ["Liberals’ Movement of the Republic of Lithuania",
      "Liberal Movement",
      "Liberal Movement of the Republic of Lithuania"]],

    ["LITH", 2519, ["FRONT"],
     ["Fronto Partija"],
     ["Front Party"]],

    ["LITH", 2520, ["DK"],
     ["Drąsos Kelias"],
     ["The Way of Courage"]],

    ["LITH", 2521, ["LCP"],
     ["Lietuvos Centro Partija"],
     ["Lithuanian Centre Party"]],

    ["LITH", 2522, ["LZP"],
     ["Lietuvos žaliųjų partija"],
     ["Lithuanian Green Party"]],

    ["LITH", 2523, ["VKM-AMT"],
     ["Visuomeninis rinkimų komitetas “Aušros Maldeikienės traukinys”"],
     ["Public election committee “Aušra Maldeikienė’s Train”"]],

    ["LITH", 2524, ["LP"],
     ["Laisvės partija"],
     ["Freedom Party"]],

    ["LITH", 2525, ["PPNA"],
     ["Nemuno aušra"],
     ["Dawn of Nemunas"]],

    ["LITH", 2526, ["DSVL"],
     ["Demokratų sąjunga „Vardan Lietuvos“"],
     ["Union of Democrats “For Lithuania”"]],

    ["LITH", 2527, ["NS"],
     ["Nacionalinis susivienijimas"],
     ["National Alliance"]],

    ["LITH", 2528, ["PLT"],
     ["Laisvė ir Teisingumas"],
     ["Freedom and Justice"]],
]

all_party_tables.append(parties_LITH)

In [26]:
# polish parties
parties_POL = [
    ["POL", 2601, ["SLD", "NL"],
     ["Sojusz Lewicy Demokratycznej",
      "Nowa Lewica"],
     ["Alliance of Democratic Left",
      "Democratic Left Alliance",
      "New Left"]],

    ["POL", 2602, ["UP"],
     ["Unia Pracy"],
     ["Union of Labour"]],

    ["POL", 2603, ["PO"],
     ["Platforma Obywatelska"],
     ["Civic Platform"]],

    ["POL", 2604, ["S"],
     ["Samoobrona RP"],
     ["Self Defense of the Polish Republic"]],

    ["POL", 2605, ["PiS"],
     ["Prawo i Sprawiedliwość"],
     ["Law and Justice Party"]],

    ["POL", 2606, ["PSL"],
     ["Polskie Stronnictwo Ludowe"],
     ["Polish People’s Party"]],

    ["POL", 2607, ["LPR"],
     ["Liga Polskich Rodzin"],
     ["League of Polish Families"]],

    ["POL", 2608, ["AWSP"],
     ["Koalicja Akcja Wyborcza Solidarność Prawicy"],
     ["Coalition Electoral Action Solidarity of the Right"]],

    ["POL", 2609, ["UW", "PD"],
     ["Unia Wolności",
      "Partia Demokratyczna"],
     ["Freedom Union",
      "Democratic Party"]],

    ["POL", 2611, ["SDPL"],
     ["Socjaldemokracja Polska"],
     ["Social Democracy of Poland"]],

    ["POL", 2612, ["SD"],
     ["Stronnictwo Demokratyczne"],
     ["Democratic Party"]],

    ["POL", 2613, ["RP"],
     ["Twój Ruch (Ruch Palikota)"],
     ["Your Movement (Palikot’s Movement)"]],

    ["POL", 2614, ["KNP"],
     ["Kongres Nowej Prawicy"],
     ["Congress of the New Right"]],

    ["POL", 2615, ["PR"],
     ["Polska Razem"],
     ["Poland Together"]],

    ["POL", 2616, ["SP"],
     ["Solidarna Polska"],
     ["United Poland"]],

    ["POL", 2617, ["Kukiz ’15", "K'15", "Kukiz"], # added K'15 and Kukiz (see wikipedia) (NOTE)
     ["Kukiz ’15"],
     ["Kukiz ’15"]],

    ["POL", 2618, ["Nowo"],
     ["Nowoczesna"],
     ["Modern"]],

    ["POL", 2619, ["Konfederacja"], # corrected to "Konfederacja" (see wiki)
     ["Konfederacja Wolność i Niepodległość"],
     ["Confederation Liberty and Independence"]],

    ["POL", 2620, ["Razem"],
     ["Lewica Razem",
      "Razem"],
     ["Left Together",
      "Together"]],

    ["POL", 2621, ["Wiosna"],
     ["Wiosna"],
     ["Spring"]],
]

all_party_tables.append(parties_POL)

In [27]:
# romanian parties
parties_RO = [
    ["ROM", 2701, ["PSD"],
     ["Partidul Social Democrat"],
     ["Social Democratic Party"]],

    ["ROM", 2702, ["PUR", "PC"],
     ["Partidul Umanist din Romania",
      "Partidul Conservator"],
     ["Romanian Humanist Party",
      "Conservative Party"]],

    ["ROM", 2703, ["PRM"],
     ["Partidul România Mare"],
     ["Party of Great Romania"]],

    ["ROM", 2704, ["PDL"],
     ["Partidul Democrat"],
     ["Democratic Party"]],

    ["ROM", 2705, ["PNL"],
     ["Partidul Național Liberal"],
     ["National Liberal Party"]],

    ["ROM", 2706, ["UDMR"],
     ["Uniunea Democrată Maghiară din România"],
     ["Hungarian Democratic Union of Romania"]],

    ["ROM", 2707, ["CDR 2000"],
     ["Convenția Democrată Română"],
     ["Romanian Democratic Convention 2000"]],

    ["ROM", 2708, ["FDGR"],
     ["Forumul Democrat al Germanilor din România"],
     ["Democratic Forum of Germans of Romania"]],

    ["ROM", 2709, ["UNPR"],
     ["Uniunea Națională pentru Progresul României"],
     ["National Union for the Progress of Romania"]],

    ["ROM", 2710, ["PP-DD"],
     ["Partidul Poporului—Dan Diaconescu"],
     ["People’s Party—Dan Diaconescu"]],

    ["ROM", 2711, ["PMP"],
     ["Partidul Mișcarea Populară"],
     ["People’s Movement Party"]],

    ["ROM", 2712, ["PLR", "ALDE"],
     ["Partidul Liberal Reformator",
      "Alianța Liberalilor și Democraților"],
     ["Liberal Reformist Party",
      "Alliance of Liberals and Democrats"]],

    ["ROM", 2713, ["USR"],
     ["Uniunea Salvați România"],
     ["Save Romania Union"]],

    ["ROM", 2714, ["PRO"],
     ["PRO România"],
     ["PRO Romania"]],

    ["ROM", 2715, ["AUR"],
     ["Alianța pentru Unirea Românilor"],
     ["Alliance for the Union of Romanians"]],

    ["ROM", 2716, ["SOS RO"],
     ["SOS România"],
     ["S.O.S. Romania"]],

    ["ROM", 2717, ["POT"],
     ["Partidul Oamenilor Tineri"],
     ["Party of Young People"]],
]

all_party_tables.append(parties_RO)

In [28]:
# slovakian parties
parties_SLO = [
    ["SLO", 2801, ["LS-HZDS"],
     ["Ľudová strana–Hnutie za demokratické Slovensko"],
     ["People’s Party–Movement for a Democratic Slovakia"]],

    ["SLO", 2802, ["SDKU", "SDKU-DS"],
     ["Slovenská demokratická a kresťanská únia",
      "Slovenská demokratická a kresťanská únia–Demokratická strana"],
     ["Slovak Democratic and Christian Union",
      "Slovak Democratic and Christian Union–Democratic Party"]],

    ["SLO", 2803, ["Smer", "Smer-SD"],
     ["Strana Smer–Tretia Cesta",
      "Smer–sociálna demokracia"],
     ["Direction–Third Way",
      "Direction–Social Democracy"]],

    ["SLO", 2804, ["SMK", "SMK-MKP"],
     ["Strana maďarskej koalície",
      # (orig. CHES "Strana madarskej koalície—Magyar") GPT removed Magyar, is that correct (NOTE)
      "Koalícia Pártja"],
     ["Party of the Hungarian Coalition",
      "Hungarian Coalition"]],

    ["SLO", 2805, ["KDH"],
     ["Kresťanskodemokratické hnutie"],
     ["Christian Democratic Movement"]],

    ["SLO", 2806, ["ANO"],
     ["Aliancia Nového Občana"],
     ["Alliance for a New Citizen"]],

    ["SLO", 2807, ["KSS"],
     ["Komunistická strana Slovenska"],
     ["Communist Party of Slovakia"]],

    ["SLO", 2808, ["PSNS"],
     ["Pravá Slovenská národná strana"],
     ["Right Slovak National Party"]],

    ["SLO", 2809, ["SNS"],
     ["Slovenská národná strana"],
     ["Slovak National Party"]],

    ["SLO", 2810, ["HZD"],
     ["Hnutie za demokraciu"],
     ["Movement for Democracy"]],

    ["SLO", 2811, ["SF"],
     ["Slobodné fórum"],
     ["Free Forum"]],

    ["SLO", 2812, ["SaS"],
     ["Sloboda a Solidarita"],
     ["Freedom and Solidarity"]],

    ["SLO", 2813, ["Most"],
     ["Most-Híd"],
     ["Bridge"]],

    ["SLO", 2814, ["OLaNO"],
     ["Obyčajní Ľudia a nezávislé osobnosti"],
     ["Ordinary People and Independent Personalities"]],

    ["SLO", 2815, ["NOVA"],
     ["Nová Väčšina"],
     ["New Majority"]],

    ["SLO", 2816, ["Sieť"],
     ["Sieť"],
     ["Net / Network"]],

    ["SLO", 2817, ["LSNS"],
     ["Ľudová strana Naše Slovensko (Marian Kotleba)"],
     ["People’s Party—Our Slovakia"]],

    ["SLO", 2818, ["Sme Rodina"],
     ["Sme Rodina–Boris Kollár"],
     ["We are family—Boris Kollár"]],

    ["SLO", 2819, ["PS"],
     ["Progresívne Slovensko"],
     ["Progressive Slovakia"]],

    ["SLO", 2820, ["SPOLU"],
     ["SPOLU–občianska demokracia"],
     ["TOGETHER—Civic Democracy"]],

    ["SLO", 2821, ["Za Ľudí", "ZĽ"], # (orig. CHES "Za Ludi") is GPT correct? (NOTE)
     # added "ZĽ" (see wiki)
     ["Za Ľudí (Andrej Kiska)"],
     ["For the People"]],

    ["SLO", 2822, ["Aliancia"],
     ["Aliancia–Szövetség"],
     ["Alliance"]],

    ["SLO", 2823, ["Hlas"],
     ["Hlas–sociálna demokracia"],
     ["Voice—Social Democracy"]],

    ["SLO", 2824, ["Republika"],
     ["Hnutie Republika"],
     ["Republic Movement"]],
]

all_party_tables.append(parties_SLO)

In [29]:
# slovenian parties
parties_SLE = [
    ["SLE", 2901, ["LDS"],
     ["Liberalna demokracija Slovenije"],
     ["Liberal Democracy of Slovenia"]],

    ["SLE", 2902, ["SDS"],
     ["Socialdemokratska stranka Slovenije",
      "Slovenska demokratska stranka"],
     ["Social Democratic Party of Slovenia",
      "Slovenian Democratic Party"]],

    ["SLE", 2903, ["ZLSD", "SD"],
     ["Združena lista socialnih demokratov",
      "Socialni Demokrati"],
     ["United List of Social Democrats",
      "Social Democrats"]],

    ["SLE", 2904, ["SLS-SKD", "SLS"],
     ["Slovenska ljudska stranka"],
     ["Slovenian People’s Party–Slovenian Christian Democrats",
      "Slovenian People’s Party"]],

    ["SLE", 2905, ["NSi", "NSI"], # (orig. CHES "NSI") but GPT corrected truthfully to "NSi" (NOTE)
     ["Nova Slovenija–Krščanska ljudska stranka"],
     ["New Slovenia–Christian People’s Party"]],

    ["SLE", 2906, ["DeSUS"],
     ["Demokratična stranka upokojencev Slovenije"],
     ["Democratic Party of Pensioners of Slovenia"]],

    ["SLE", 2907, ["SNS"],
     ["Slovenska nacionalna stranka"],
     ["Slovenian National Party"]],

    ["SLE", 2908, ["SMS"],
     ["Stranka Mladih Slovenije"],
     ["Party of the Slovenia Youth"]],

    ["SLE", 2909, ["AS"],
     ["Aktivna Slovenija"],
     ["Active Slovenia"]],

    ["SLE", 2910, ["Zares"],
     ["Zares"],
     ["Zares—For Real"]],

    ["SLE", 2911, ["SMC"],
     ["Stranka Mira Cerarja"],
     ["Party of Miro Cerar"]],

    ["SLE", 2912, ["ZL", "Levica"],
     ["Združena Levica",
      "Levica"],
     ["United Left",
      "The Left"]],

    ["SLE", 2913, ["ZaAB", "AB"],
     ["Zavezništvo Alenke Bratušek",
      "Stranka Alenke Bratušek"],
     ["Alliance of Alenka Bratušek",
      "Party of Alenka Bratušek"]],

    ["SLE", 2914, ["PS"],
     ["Pozitivna Slovenija"],
     ["Positive Slovenia"]],

    ["SLE", 2915, ["LMS"],
     ["Lista Marjana Šarca"],
     ["List of Marjan Šarec"]],

    ["SLE", 2916, ["GS"],
     ["Gibanje Svoboda"],
     ["Freedom Movement"]],

    ["SLE", 2917, ["Vesna"],
     ["VESNA–zelena stranka"],
     ["Spring—Green party"]],

    ["SLE", 2918, ["Resni.ca"],
     ["Resni.ca"],
     ["Truth"]], # (orig. CHES "Tru.th") GPT corrected to "Truth" is that correct? (NOTE)

#    ["SLE", 2919, ["SLS"],                  # NOTE this is a duplicate of party with id "2904" so merge these together in raw CHES!!!
#     ["Slovenska ljudska stranka"],         # NOTE: otherwise in "enrich_memberships_with_CHES" MEP Franc Bogovič maps to both parties at the same time (!!)
#     ["Slovenian People’s Party"]],
]

all_party_tables.append(parties_SLE)

NOTE: Apparently, the MEP Franc Bogovič gets mapped ambiguously to two parties at the same time -> cannot be. This is because we have two entries in the CHES dataset (i.e. two different IDs) for the same party, namely
the "Slovenian People’s Party", where the party with ID 2919 only has a single datapoint in 2024 and the other party reflects the rest of the corresponding datapoints.
Consequence: Merge these two together (in the raw CHES also) to remove this ambiguity 

In [30]:
# croatian parties
parties_CRO =[
    ["CRO", 3101, ["HDZ"],
     ["Hrvatska Demokratska Zajednica"],
     ["Croatian Democratic Union"]],

    ["CRO", 3102, ["SDP"],
     ["Socijaldemokratska Partija Hrvatske"],
     ["Social Democratic Party of Croatia"]],

    ["CRO", 3103, ["HSS"],
     ["Hrvatska Seljačka Stranka"],
     ["Croatian Peasant Party"]],

    ["CRO", 3104, ["HSLS"],
     ["Hrvatska Socijalno Liberalna Stranka"],
     ["Croatian Social Liberal Party"]],

    ["CRO", 3105, ["HNS"],
     ["Hrvatska Narodna Stranka–Liberalni Demokrati"],
     ["Croatian People’s Party—Liberal Democrats"]],

    ["CRO", 3106, ["IDS"],
     ["Istarski Demokratski Sabor"],
     ["Istrian Democratic Assembly"]],

    ["CRO", 3107, ["HDSSB"],
     ["Hrvatski Demokrtski Sabor Slavonije i Baranje"],
     ["Croatian Democratic Assembly of Slavonija and Baranja"]],

    ["CRO", 3108, ["HSU"],
     ["Hrvatska stranka umirovljenika"],
     ["Croatian Party of Pensioners"]],

    ["CRO", 3109, ["HSP"],
     ["Hrvatska Stranka Prava"],
     ["Croatian Party of Rights"]],

    ["CRO", 3110, ["SDSS"],
     ["Samostalna demokratska srpska stranka"],
     ["Independent Democratic Serb Party"]],

    ["CRO", 3112, ["HL-SR"],
     ["Hrvatski Laburisti–Stranka Rada"],
     ["Croatian Labourists–Labour Party"]],

    ["CRO", 3113, ["HSP-AS"],
     ["Hrvatska stranka prava dr. Ante Starčević"],
     ["Croatian Party of Rights dr. Ante Starčević"]],

    ["CRO", 3114, ["ORaH"],
     ["Održivi Razvoj Hrvatske"],
     ["Croatian Sustainable Development"]],

    ["CRO", 3115, ["Most"],
     ["Most nezavisnih lista"],
     ["Bridge of Independent Lists"]],

    ["CRO", 3116, ["ZZ"],
     ["Živi zid"],
     ["Human Shield"]],

    ["CRO", 3117, ["MB 365"],
     ["Milan Bandić 365–Stranka rada i solidarnosti"],
     ["Milan Bandić 365—The Party of Labour and Solidarity"]],

    ["CRO", 3118, ["NS-R"],
     ["Narodna stranka–Reformisti"],
     ["People’s Party–Reformists"]],

    ["CRO", 3119, ["HKS"],
     ["Hrvatska konzervativna stranka"],
     ["Croatian Conservative Party"]],

    ["CRO", 3120, ["M!"],
     ["Možemo!"],
     ["We Can!"]],

    ["CRO", 3121, ["Centar"],
     ["Centar"],
     ["Centre"]],

    ["CRO", 3122, ["DPS"],
     ["Domovinski pokret"],
     ["Homeland Movement"]],
]

all_party_tables.append(parties_CRO)

In [31]:
# parties of Malta
parties_MAL = [
    ["MAL", 3701, ["PL"],
     ["Partit Laburista"],
     ["Labour Party"]],

    ["MAL", 3702, ["PN"],
     ["Partit Nazzjonalista"],
     ["Nationalist Party"]],
]

all_party_tables.append(parties_MAL)

In [32]:
# parties of Luxembourg
parties_LUX =[
    ["LUX", 3801, ["CSV"],
     ["Chrëschtlech Sozial Vollekspartei"],
     ["Christian Social People’s Party"]],

    ["LUX", 3802, ["Gréng", "Greng"], # (orig. CHES "Greng") GPT corrected to "Gréng" (NOTE)
     ["Déi Gréng"],
     ["The Greens"]],

    ["LUX", 3803, ["DP"],
     ["Demokratesch Partei"],
     ["Democratic Party"]],

    ["LUX", 3804, ["LSAP"],
     ["Lëtzebuerger Sozialistesch Aarbechterpartei"],
     ["Luxembourg Socialist Workers’ Party"]],

    ["LUX", 3805, ["ADR"],
     ["Alternativ Demokratesch Reformpartei"],
     ["Alternative Democratic Reform Party"]],

    ["LUX", 3806, ["DL"],
     ["Déi Lénk"],
     ["The Left"]],

    ["LUX", 3807, ["PPLU"],
     ["Piratepartei Lëtzebuerg"],
     ["Pirate Party Luxembourg"]],

    ["LUX", 3808, ["KPL"],
     ["Kommunistesch Partei Lëtzebuerg"],
     ["Communist Party of Luxembourg"]],

    ["LUX", 3809, ["PID"],
     ["Partei fir Integral Demokratie"],
     ["Party for Full Democracy"]],
]

all_party_tables.append(parties_LUX)

In [33]:
# parties of Cyprus
parties_CYP = [
    ["CYP", 4001, ["DISY"],
     ["Dimokratikós Sinagermós"],
     ["Democratic Rally"]],

    ["CYP", 4002, ["Evroko"],
     ["Evropaïkó Kómma"],
     ["European Party"]],

    ["CYP", 4003, ["AKEL"],
     ["Anorthotikó Kómma Ergazómenou Laoú"],
     ["Progressive Party of Working People"]],

    ["CYP", 4004, ["DIKO"],
     ["Dimokratikó Kómma"],
     ["Democratic Party"]],

    ["CYP", 4005, ["EDEK"],
     ["Kinima Sosialdimokraton EDEK"],
     ["Movement for Social Democracy EDEK"]],

    ["CYP", 4006, ["KOP"],
     ["Kinima Oikologon Perivallontiston"],
     ["Ecological and Environmental Movement"]],

    ["CYP", 4007, ["SYM"],
     ["Symmachía Politón"],
     ["Citizens’ Alliance"]],

    ["CYP", 4008, ["Kinhma"],
     ["Kinima Allilengyi"],
     ["Solidarity Movement"]],

    ["CYP", 4009, ["ELAM"],
     ["Ethniko Laiko Metopo"],
     ["National Popular Front"]],
]

all_party_tables.append(parties_CYP)

Now lets put it all together

In [34]:
flattened_list = [row for table in all_party_tables for row in table]
all_parties_df = pd.DataFrame(flattened_list, columns = columns)
all_parties_df.head()

,Country,Party ID,Party Abbrev,Party Name,Party Name (English)
0,BE,102,[PS],[Parti Socialiste],[Socialist Party]
1,BE,103,"[Vooruit/SP, SPA]","[Vooruit, Socialistische Partij, Socialistisch...","[Onward, Socialist Party, Socialist Party Diff..."
2,BE,104,[ECOLO],[Ecolo],[Ecolo]
3,BE,105,"[AGALEV, Groen]","[Agalev, Groen!]","[Agalev, Green!]"
4,BE,106,"[PRL, MR]","[Parti Réformateur Libéral, Mouvement Réformat...","[Liberal Reformist Party, Reformist Movement]"


In [35]:
# map the country codes with ISO codes (used by the EU OpenData Portal)
all_parties_df["Country"] = all_parties_df["Country"].map(country_codes_CHES)
all_parties_df["Country"] = all_parties_df["Country"].map(country_codes_ISO)
all_parties_df.head()

,Country,Party ID,Party Abbrev,Party Name,Party Name (English)
0,BEL,102,[PS],[Parti Socialiste],[Socialist Party]
1,BEL,103,"[Vooruit/SP, SPA]","[Vooruit, Socialistische Partij, Socialistisch...","[Onward, Socialist Party, Socialist Party Diff..."
2,BEL,104,[ECOLO],[Ecolo],[Ecolo]
3,BEL,105,"[AGALEV, Groen]","[Agalev, Groen!]","[Agalev, Green!]"
4,BEL,106,"[PRL, MR]","[Parti Réformateur Libéral, Mouvement Réformat...","[Liberal Reformist Party, Reformist Movement]"


Load the original (raw) CHES file and perform sanity checks

In [36]:
# load the raw ches data
survey_df = pd.read_csv(PATH_ORIGINAL_CHES_RAW_CSV)

survey_df.head()

,year,country,eastwest,eumember,party_id,party,cmp_id,vote,seat,electionyear,epvote,family,govt,lrgen,lrecon,lrecon_salience,lrecon_dissent,lrecon_blur,galtan,galtan_salience,galtan_dissent,galtan_blur,eu_position,eu_salience,eu_dissent,eu_blur,spendvtax,spendvtax_salience,deregulation,dereg_salience,redistribution,redist_salience,econ_interven,civlib_laworder,civlib_salience,sociallifestyle,social_salience,womens_rights,lgbtq_rights,samesex_marriage,religious_principles,relig_salience,immigrate_policy,immigrate_salience,immigrate_dissent,multiculturalism,multicult_salience,multicult_dissent,nationalism,nationalism_salience,ethnic_minorities,ethnic_salience,urban_rural,urban_salience,environment,enviro_salience,climate_change,climate_change_salience,protectionism,regions,region_salience,international_security,international_salience,us,us_salience,eu_benefit,eu_ep,eu_fiscal,eu_intmark,eu_employ,eu_budgets,eu_agri,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_turkey,eu_russia,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,executive_power,judicial_independence,mip_one,mip_two,mip_three,chesversion
0,1999,1,1,1,102,PS,21322.0,10.2,12.700000,1999,9.59,5,1.0,3.111111,2.625000,NaN,NaN,NaN,3.875000,NaN,NaN,NaN,6.666667,4.722222,1.388889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.333333,6.666667,NaN,6.555555,NaN,NaN,6.625,5.888889,5.666667,6.555555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1,2002,1,1,1,102,PS,21322.0,10.2,12.700000,1999,9.59,5,1.0,3.350000,2.500000,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,6.090000,4.533333,2.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,NaN,4.100000,6.550000,NaN,4.88,5.850,5.000000,6.300000,6.330000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
2,2006,1,1,1,102,PS,21322.0,13.0,16.700001,2003,13.50,5,1.0,3.500000,3.170000,NaN,NaN,NaN,2.830000,NaN,NaN,NaN,5.710000,4.300000,3.430000,NaN,2.670000,6.500000,2.330,6.83,2.000000,8.000000,NaN,3.670000,6.500000,2.330000,5.670000,NaN,NaN,NaN,1.330000,5.670000,2.5,6.330000,NaN,4.20,NaN,NaN,3.17,4.33,3.170000,6.330000,4.400000,3.000000,NaN,NaN,NaN,NaN,NaN,6.170000,7.000000,NaN,NaN,2.67,3.5,NaN,6.500000,NaN,4.000000,NaN,NaN,NaN,6.670,NaN,NaN,5.830000,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
3,2010,1,1,1,102,PS,21322.0,13.7,17.299999,2010,10.88,5,1.0,2.500000,2.571429,NaN,NaN,NaN,2.857143,NaN,NaN,NaN,6.200000,4.523810,2.461539,NaN,1.692308,7.615385,2.375,4.50,1.818182,7.583333,NaN,2.636364,4.727273,1.818182,5.090909,NaN,NaN,NaN,1.454545,2.545455,2.0,6.083333,NaN,3.25,5.545454,NaN,NaN,NaN,2.444444,5.333333,4.000000,2.142857,4.916666,4.833334,NaN,NaN,NaN,5.583334,5.083334,6.090909,3.222222,NaN,NaN,1.133333,6.545455,NaN,5.083333,NaN,NaN,NaN,6.625,NaN,NaN,6.333333,5.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
4,2014,1,1,1,102,PS,21322.0,11.7,15.300000,2014,10.70,5,0.5,2.600000,2.400000,8.4,NaN,NaN,3.400000,3.2,NaN,NaN,6.000000,3.200000,3.000000,NaN,2.400000,NaN,2.000,NaN,1.800000,NaN,1.75,2.200000,NaN,1.600000,NaN,NaN,NaN,NaN,1.000000,NaN,2.8,NaN,NaN,2.00,NaN,NaN,2.25,NaN,2.000000,NaN,3.333333,NaN,3.800000,NaN,NaN,NaN,NaN,6.600000,NaN,5.800000,NaN,NaN,NaN,1.200000,6.400000,NaN,5.000000,NaN,4.2,NaN,6.250,NaN,NaN,6.000000,5.50,NaN,NaN,NaN,NaN,2.6,2.25,NaN,NaN,NaN,13.0,14.0,17.0,2025.1


In [37]:
# rename columns for convenience
meta_df = all_parties_df.rename(columns={
    "Country" : "country_code",
    "Party ID" : "party_id",
    "Party Abbrev" : "party_abbrev",
    "Party Name" : "party_name",
    "Party Name (English)" : "party_name_en"
})

survey_df = survey_df.rename(columns={
    "country" : "country_code",
    "party" : "party_label"
})
meta_df.head()

,country_code,party_id,party_abbrev,party_name,party_name_en
0,BEL,102,[PS],[Parti Socialiste],[Socialist Party]
1,BEL,103,"[Vooruit/SP, SPA]","[Vooruit, Socialistische Partij, Socialistisch...","[Onward, Socialist Party, Socialist Party Diff..."
2,BEL,104,[ECOLO],[Ecolo],[Ecolo]
3,BEL,105,"[AGALEV, Groen]","[Agalev, Groen!]","[Agalev, Green!]"
4,BEL,106,"[PRL, MR]","[Parti Réformateur Libéral, Mouvement Réformat...","[Liberal Reformist Party, Reformist Movement]"


In [38]:
# map CHES country numbers to country ISO codes
survey_df["country_code"] = survey_df["country_code"].map(country_codes_ISO)
survey_df.head()

,year,country_code,eastwest,eumember,party_id,party_label,cmp_id,vote,seat,electionyear,epvote,family,govt,lrgen,lrecon,lrecon_salience,lrecon_dissent,lrecon_blur,galtan,galtan_salience,galtan_dissent,galtan_blur,eu_position,eu_salience,eu_dissent,eu_blur,spendvtax,spendvtax_salience,deregulation,dereg_salience,redistribution,redist_salience,econ_interven,civlib_laworder,civlib_salience,sociallifestyle,social_salience,womens_rights,lgbtq_rights,samesex_marriage,religious_principles,relig_salience,immigrate_policy,immigrate_salience,immigrate_dissent,multiculturalism,multicult_salience,multicult_dissent,nationalism,nationalism_salience,ethnic_minorities,ethnic_salience,urban_rural,urban_salience,environment,enviro_salience,climate_change,climate_change_salience,protectionism,regions,region_salience,international_security,international_salience,us,us_salience,eu_benefit,eu_ep,eu_fiscal,eu_intmark,eu_employ,eu_budgets,eu_agri,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_turkey,eu_russia,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,executive_power,judicial_independence,mip_one,mip_two,mip_three,chesversion
0,1999,BEL,1,1,102,PS,21322.0,10.2,12.700000,1999,9.59,5,1.0,3.111111,2.625000,NaN,NaN,NaN,3.875000,NaN,NaN,NaN,6.666667,4.722222,1.388889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.333333,6.666667,NaN,6.555555,NaN,NaN,6.625,5.888889,5.666667,6.555555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1,2002,BEL,1,1,102,PS,21322.0,10.2,12.700000,1999,9.59,5,1.0,3.350000,2.500000,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,6.090000,4.533333,2.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,NaN,4.100000,6.550000,NaN,4.88,5.850,5.000000,6.300000,6.330000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
2,2006,BEL,1,1,102,PS,21322.0,13.0,16.700001,2003,13.50,5,1.0,3.500000,3.170000,NaN,NaN,NaN,2.830000,NaN,NaN,NaN,5.710000,4.300000,3.430000,NaN,2.670000,6.500000,2.330,6.83,2.000000,8.000000,NaN,3.670000,6.500000,2.330000,5.670000,NaN,NaN,NaN,1.330000,5.670000,2.5,6.330000,NaN,4.20,NaN,NaN,3.17,4.33,3.170000,6.330000,4.400000,3.000000,NaN,NaN,NaN,NaN,NaN,6.170000,7.000000,NaN,NaN,2.67,3.5,NaN,6.500000,NaN,4.000000,NaN,NaN,NaN,6.670,NaN,NaN,5.830000,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
3,2010,BEL,1,1,102,PS,21322.0,13.7,17.299999,2010,10.88,5,1.0,2.500000,2.571429,NaN,NaN,NaN,2.857143,NaN,NaN,NaN,6.200000,4.523810,2.461539,NaN,1.692308,7.615385,2.375,4.50,1.818182,7.583333,NaN,2.636364,4.727273,1.818182,5.090909,NaN,NaN,NaN,1.454545,2.545455,2.0,6.083333,NaN,3.25,5.545454,NaN,NaN,NaN,2.444444,5.333333,4.000000,2.142857,4.916666,4.833334,NaN,NaN,NaN,5.583334,5.083334,6.090909,3.222222,NaN,NaN,1.133333,6.545455,NaN,5.083333,NaN,NaN,NaN,6.625,NaN,NaN,6.333333,5.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
4,2014,BEL,1,1,102,PS,21322.0,11.7,15.300000,2014,10.70,5,0.5,2.600000,2.400000,8.4,NaN,NaN,3.400000,3.2,NaN,NaN,6.000000,3.200000,3.000000,NaN,2.400000,NaN,2.000,NaN,1.800000,NaN,1.75,2.200000,NaN,1.600000,NaN,NaN,NaN,NaN,1.000000,NaN,2.8,NaN,NaN,2.00,NaN,NaN,2.25,NaN,2.000000,NaN,3.333333,NaN,3.800000,NaN,NaN,NaN,NaN,6.600000,NaN,5.800000,NaN,NaN,NaN,1.200000,6.400000,NaN,5.000000,NaN,4.2,NaN,6.250,NaN,NaN,6.000000,5.50,NaN,NaN,NaN,NaN,2.6,2.25,NaN,NaN,NaN,13.0,14.0,17.0,2025.1


In [39]:
# check for "NaN" (the string) in column party_label (since this seems to occur)
display(survey_df[survey_df['party_label'].isnull()])
# all these rows should have the party label "NA" (or "TB-LNNK"), so fix this
fixed_survey = survey_df.fillna("NA")
display(fixed_survey[fixed_survey['party_label'].isnull()])
survey_df = fixed_survey

,year,country_code,eastwest,eumember,party_id,party_label,cmp_id,vote,seat,electionyear,epvote,family,govt,lrgen,lrecon,lrecon_salience,lrecon_dissent,lrecon_blur,galtan,galtan_salience,galtan_dissent,galtan_blur,eu_position,eu_salience,eu_dissent,eu_blur,spendvtax,spendvtax_salience,deregulation,dereg_salience,redistribution,redist_salience,econ_interven,civlib_laworder,civlib_salience,sociallifestyle,social_salience,womens_rights,lgbtq_rights,samesex_marriage,religious_principles,relig_salience,immigrate_policy,immigrate_salience,immigrate_dissent,multiculturalism,multicult_salience,multicult_dissent,nationalism,nationalism_salience,ethnic_minorities,ethnic_salience,urban_rural,urban_salience,environment,enviro_salience,climate_change,climate_change_salience,protectionism,regions,region_salience,international_security,international_salience,us,us_salience,eu_benefit,eu_ep,eu_fiscal,eu_intmark,eu_employ,eu_budgets,eu_agri,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_turkey,eu_russia,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,executive_power,judicial_independence,mip_one,mip_two,mip_three,chesversion
1076,2010,LVA,0,1,2406,NaN,87723.0,13.88,14.0,2011,7.50,1,1.0,7.750000,5.111111,NaN,NaN,NaN,6.666667,NaN,NaN,NaN,4.444445,5.185185,5.250,NaN,5.000000,4.428571,4.714286,4.428571,5.555555,4.375000,NaN,5.571429,4.333333,8.125000,5.375,NaN,NaN,NaN,3.625,3.0,6.714286,8.571428,NaN,9.250000,7.875000,NaN,NaN,NaN,9.222222,8.333333,6.250000,6.428571,5.000000,5.285714,NaN,NaN,NaN,4.500000,5.2,3.25,5.0,NaN,NaN,1.444444,4.857143,NaN,4.375000,NaN,NaN,NaN,5.500000,NaN,NaN,5.250000,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1077,2014,LVA,0,1,2406,NaN,87723.0,16.60,17.0,2014,14.30,1,1.0,8.300000,5.888889,6.666666,NaN,NaN,8.111111,6.666666,NaN,NaN,5.700000,6.800000,3.875,NaN,5.625000,NaN,4.875000,NaN,5.875000,NaN,6.625000,6.125000,NaN,7.444445,NaN,NaN,NaN,NaN,5.375,NaN,8.714286,NaN,NaN,8.000000,NaN,NaN,9.750000,NaN,8.777778,NaN,6.125000,NaN,6.333334,NaN,NaN,NaN,NaN,6.142857,NaN,2.25,NaN,NaN,NaN,1.111111,4.875000,NaN,5.750000,NaN,4.222222,NaN,6.714286,NaN,NaN,6.250000,3.8,NaN,NaN,NaN,NaN,5.000000,6.888889,NaN,NaN,NaN,12.0,10.0,9.0,2025.1
1078,2019,LVA,0,1,2406,NaN,87723.0,11.08,13.0,2018,16.40,1,1.0,8.454545,6.090909,4.818182,3.5,4.666666,8.090909,7.800000,3.857143,2.333333,4.727273,6.363637,4.500,3.75,5.500000,NaN,5.000000,NaN,6.272727,3.818182,5.916666,7.833333,NaN,9.000000,NaN,NaN,NaN,NaN,5.600,NaN,9.666667,9.000000,2.090909,9.100000,7.818182,1.0,9.916667,NaN,9.666667,NaN,6.333333,NaN,5.363637,2.818182,NaN,NaN,7.4,5.818182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.333333,NaN,4.200000,NaN,5.833333,NaN,1.416667,3.833333,NaN,NaN,7.916666,5.909091,4.200,3.666667,5.000000,7.454546,NaN,NaN,NaN,NaN,NaN,2025.1
1079,2024,LVA,0,1,2406,NaN,87723.0,9.40,13.0,2022,22.32,1,NaN,7.842105,5.750000,6.250000,1.8,4.454546,9.050000,8.157895,1.400000,2.846154,5.380952,6.800000,3.625,2.70,5.111111,NaN,4.200000,NaN,5.687500,5.400000,NaN,7.714286,NaN,NaN,NaN,5.3,7.142857,8.8,6.000,NaN,9.157895,8.157895,1.066667,9.117647,6.812500,NaN,8.750000,NaN,8.200000,NaN,6.125000,NaN,4.750000,4.833334,6.625,4.25,7.2,6.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.400000,NaN,NaN,NaN,NaN,NaN,NaN,4.166667,NaN,0.5,NaN,6.666666,3.875,4.428571,6.000000,NaN,5.857143,2.8,NaN,NaN,NaN,2025.1


,year,country_code,eastwest,eumember,party_id,party_label,cmp_id,vote,seat,electionyear,epvote,family,govt,lrgen,lrecon,lrecon_salience,lrecon_dissent,lrecon_blur,galtan,galtan_salience,galtan_dissent,galtan_blur,eu_position,eu_salience,eu_dissent,eu_blur,spendvtax,spendvtax_salience,deregulation,dereg_salience,redistribution,redist_salience,econ_interven,civlib_laworder,civlib_salience,sociallifestyle,social_salience,womens_rights,lgbtq_rights,samesex_marriage,religious_principles,relig_salience,immigrate_policy,immigrate_salience,immigrate_dissent,multiculturalism,multicult_salience,multicult_dissent,nationalism,nationalism_salience,ethnic_minorities,ethnic_salience,urban_rural,urban_salience,environment,enviro_salience,climate_change,climate_change_salience,protectionism,regions,region_salience,international_security,international_salience,us,us_salience,eu_benefit,eu_ep,eu_fiscal,eu_intmark,eu_employ,eu_budgets,eu_agri,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_turkey,eu_russia,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,executive_power,judicial_independence,mip_one,mip_two,mip_three,chesversion


In [40]:
# check for ches survey if each combo of country code and party_id is also present in the meta df
allowed_country_party = set(zip(meta_df["country_code"], meta_df["party_id"]))
survey_df["country_party_pair_ok"] = list(zip(survey_df["country_code"], survey_df["party_id"]))  \
    # tuples per row
survey_df["country_party_pair_ok"] = survey_df["country_party_pair_ok"].apply(lambda t: t in allowed_country_party)

print(survey_df["country_party_pair_ok"].value_counts(dropna=False))
bad_pairs = survey_df.loc[~survey_df["country_party_pair_ok"], ["country_code", "party_id", "party_label"]].drop_duplicates().sort_values(["country_code", "party_id", "party_label"])
print(bad_pairs)

country_party_pair_ok
True     1434
False       7
Name: count, dtype: int64
     country_code  party_id  party_label
1440          CYP      4015         DIPA
818           FIN      1410         Liik
441           IRL       706           PD
1214          POL      2623  Polska 2050
1367          SVN      2919          SLS


In [41]:
# investigate the bad pairs
bad_pairs = survey_df.loc[~survey_df["country_party_pair_ok"], ["year", "country_code", "eastwest", "eumember", "party_id", "party_label", "electionyear", "family", "chesversion"]]
print(bad_pairs)

      year country_code  eastwest  eumember  party_id  party_label  \
441   1999          IRL         1         1       706           PD   
442   2002          IRL         1         1       706           PD   
443   2006          IRL         1         1       706           PD   
818   2024          FIN         1         1      1410         Liik   
1214  2024          POL         0         1      2623  Polska 2050   
1367  2024          SVN         0         1      2919          SLS   
1440  2024          CYP         0         1      4015         DIPA   

      electionyear  family  chesversion  
441           1997       3       2025.1  
442           2002       3       2025.1  
443           2002       3       2025.1  
818           2023       3       2025.1  
1214          2023       3       2025.1  
1367          2023       4       2025.1  
1440          2021       3       2025.1  


It seems that these parties (from the survey) are somehow missing in the CHES codebook, but all of them exist(ed):

PD - most probably the Progressive Democrats (Irish: An Páirtí Daonlathach) -> dissolved in 2009 (see [wikipedia](https://en.wikipedia.org/wiki/Progressive_Democrats]))

Liik - most probably the "Movement Now" party in Finnland (MN; Finnish: Liike Nyt, Liik; Swedish: Rörelse Nu, RN) -> founded 2019 (see [wikipedia](https://en.wikipedia.org/wiki/Movement_Now))

Poland 2050 - centre-right political party in Poland, founded in 2020 (see [wikipedia](https://en.wikipedia.org/wiki/Poland_2050))

DIPA - most probably the "Democratic Alignment" / "Democratic Fron" Party (centrists) in Cyprus, created after disagreements with Democratic Party (DIKO) founded in 2018 (see [wikipedia](https://en.wikipedia.org/wiki/Democratic_Alignment_(Cyprus)))

Therefore we add them to the CHES meta data manually


In [42]:
missing_rows_meta_df = pd.DataFrame([
    {
        "country_code": "CYP",
        "party_id":4015,
        "party_abbrev": ["DIPA", "DiPa", "DEPA", "DIPO"],
        "party_name": ["Dimokratiki Parataxi"],
        "party_name_en": ["Democratic Alignment", "Democratic Front"]
    },
    {
        "country_code": "FIN",
        "party_id": 1410,
        "party_abbrev": ["Liik", "RN", "MN"],
        "party_name": ["Liike Nyt", "Rörelse Nu"],
        "party_name_en": ["Movement Now"]
    },
    {
       "country_code": "IRL",
        "party_id": 706,
        "party_abbrev": ["PD"],
        "party_name": ["An Páirtí Daonlathach"],
        "party_name_en": ["The Democratic Party", "Progressive Democrats"]
    },
    {
       "country_code": "POL",
        "party_id": 2623,
        "party_abbrev": ["PL2050", "Polska 2050"],
        "party_name": ["Polska 2050 Szymona Hołowni"],
        "party_name_en": ["Szymon Hołownia's Poland 2050"]
    },
])

meta_df = pd.concat([meta_df, missing_rows_meta_df], ignore_index=True)
# check for (introduced) duplicates (should be False)
print(meta_df.duplicated(["country_code", "party_id"]).any())

False


Now lets check again if the party labels and party ids match

In [43]:
second_keys = ["country_code", "party_id"]

second_tmp = survey_df.merge(
    meta_df[second_keys + ["party_abbrev"]],
    on=second_keys,
    how="left",
    validate="m:1"   # many survey rows -> one meta row per key
)

second_tmp["label_matches_abbrev"] = second_tmp.apply(
    lambda r: isinstance(r["party_abbrev"], list) and (r["party_label"] in r["party_abbrev"]),
    axis=1
)

# attach result back to survey (keeps original row order)
survey_df["label_matches_abbrev"] = second_tmp["label_matches_abbrev"].to_numpy()

In [44]:
# inspect the bad
second_bad = second_tmp.loc[~second_tmp["label_matches_abbrev"], second_keys + ["party_label", "party_abbrev"]]
display(second_bad.head(20))

,country_code,party_id,party_label,party_abbrev
7,BEL,103,SP,"[Vooruit/SP, SPA]"
8,BEL,103,SP,"[Vooruit/SP, SPA]"
12,BEL,103,SP/SPA,"[Vooruit/SP, SPA]"
13,BEL,103,Vooruit,"[Vooruit/SP, SPA]"
20,BEL,104,Ecolo,[ECOLO]
33,BEL,106,PRL/MR,"[PRL, MR]"
35,BEL,107,PVV/VLD,"[VLD, Open VLD, PVV]"
40,BEL,107,VLD/PVV,"[VLD, Open VLD, PVV]"
41,BEL,107,Open Vld,"[VLD, Open VLD, PVV]"
47,BEL,108,PSC/CDH,"[PSC, CDH, LE]"


Manual inspection shows, these are all equivalent labels, but yet not present in our meta file.
So lets fix this, i.e.
For every (party_id, party_label) in survey:
ensure party_label ∈ meta.party_abbrev for that party_id.

In [45]:
# collect all labels used per party in the survey
labels_by_party = (
    survey_df
    .groupby("party_id")["party_label"]
    .apply(lambda s: sorted(set(s.dropna())))
)

# attach to meta
meta2 = meta_df.set_index("party_id")
meta2["survey_labels"] = labels_by_party

#merge the lists
def merge_lists(old, new):
    old = old if isinstance(old, list) else []
    new = new if isinstance(new, list) else []
    return sorted(set(old) | set(new))

meta2["party_abbrev"] = [
    merge_lists(old, new)
    for old, new in zip(meta2["party_abbrev"], meta2["survey_labels"])
]

# clean up
meta_df = meta2.drop(columns="survey_labels").reset_index()

# verify if we did it correctly
lookup = meta_df.set_index("party_id")["party_abbrev"].to_dict()

survey_df["label_ok"] = [
    label in lookup.get(pid, [])
    for pid, label in zip(survey_df["party_id"], survey_df["party_label"])
]

print(survey_df["label_ok"].all()) # should return True
display(meta_df.head(15))

False


,party_id,country_code,party_abbrev,party_name,party_name_en
0,102,BEL,[PS],[Parti Socialiste],[Socialist Party]
1,103,BEL,"[SP, SP/SPA, SPA, Vooruit, Vooruit/SP]","[Vooruit, Socialistische Partij, Socialistisch...","[Onward, Socialist Party, Socialist Party Diff..."
2,104,BEL,"[ECOLO, Ecolo]",[Ecolo],[Ecolo]
3,105,BEL,"[AGALEV, Groen]","[Agalev, Groen!]","[Agalev, Green!]"
4,106,BEL,"[MR, PRL, PRL/MR]","[Parti Réformateur Libéral, Mouvement Réformat...","[Liberal Reformist Party, Reformist Movement]"
5,107,BEL,"[Open VLD, Open Vld, PVV, PVV/VLD, VLD, VLD/PVV]","[Open Vlaamse Liberalen en Democraten, Partij ...","[Open Flemish Liberals and Democrats, Party of..."
6,108,BEL,"[CDH, LE, PSC, PSC/CDH]","[Parti Social Chrétien, Centre Démocrate Human...","[Christian Social Party, Humanist Democratic C..."
7,109,BEL,"[CD&V, CDV, CVP]","[Christelijke Volkspartij, Christen-Democratis...","[Christian People’s Party, Christian Democrati..."
8,110,BEL,"[NVA, VU, VU/NVA]","[Volksunie, Nieuw-Vlaamse Alliantie]","[People’s Union, New Flemish Alliance]"
9,111,BEL,"[DeFl, DéFI, FDF]","[Front Démocratique des Francophones, Démocrat...","[Francophone Democratic Front, Christian Democ..."


investigate this further

In [46]:
print(survey_df["party_id"].nunique())
print(meta_df.shape[0])
# apparently, the meta_file contains some parties not reflected in the ches survey (shouldnt be a problem though)
survey_ids = set(survey_df["party_id"].dropna().unique())
meta_ids   = set(meta_df["party_id"].dropna().unique())

unused_in_survey = sorted(meta_ids - survey_ids)
print(unused_in_survey)

# inspect those parties
meta_unused = meta_df[meta_df["party_id"].isin(unused_in_survey)]
meta_unused.sort_values(["country_code", "party_id"])

482
484
[np.int64(822), np.int64(3808), np.int64(3809)]


,party_id,country_code,party_abbrev,party_name,party_name_en
146,822,ITA,[PdUP],[Partito di Unità Proletaria],[Party of Proletarian Unity for Communism]
469,3808,LUX,[KPL],[Kommunistesch Partei Lëtzebuerg],[Communist Party of Luxembourg]
470,3809,LUX,[PID],[Partei fir Integral Demokratie],[Party for Full Democracy]


In [47]:
# since meta file should only reflect parties present in the original CHES file, we omit these entries
survey_ids = set(survey_df["party_id"].dropna().unique())
meta_df = meta_df[meta_df["party_id"].isin(survey_ids)].sort_values("party_id").reset_index(drop=True)
assert set(meta_df["party_id"]) <= set(survey_df["party_id"])

Wonderful now lets store this for later use

In [ ]:
display(meta_df.head(20))

# store as parquet file
# meta_df.to_parquet(PATH_INTERMED_CHES_50_CHES_META, engine='fastparquet')

,party_id,country_code,party_abbrev,party_name,party_name_en
0,102,BEL,[PS],[Parti Socialiste],[Socialist Party]
1,103,BEL,"[SP, SP/SPA, SPA, Vooruit, Vooruit/SP]","[Vooruit, Socialistische Partij, Socialistisch...","[Onward, Socialist Party, Socialist Party Diff..."
2,104,BEL,"[ECOLO, Ecolo]",[Ecolo],[Ecolo]
3,105,BEL,"[AGALEV, Groen]","[Agalev, Groen!]","[Agalev, Green!]"
4,106,BEL,"[MR, PRL, PRL/MR]","[Parti Réformateur Libéral, Mouvement Réformat...","[Liberal Reformist Party, Reformist Movement]"
5,107,BEL,"[Open VLD, Open Vld, PVV, PVV/VLD, VLD, VLD/PVV]","[Open Vlaamse Liberalen en Democraten, Partij ...","[Open Flemish Liberals and Democrats, Party of..."
6,108,BEL,"[CDH, LE, PSC, PSC/CDH]","[Parti Social Chrétien, Centre Démocrate Human...","[Christian Social Party, Humanist Democratic C..."
7,109,BEL,"[CD&V, CDV, CVP]","[Christelijke Volkspartij, Christen-Democratis...","[Christian People’s Party, Christian Democrati..."
8,110,BEL,"[NVA, VU, VU/NVA]","[Volksunie, Nieuw-Vlaamse Alliantie]","[People’s Union, New Flemish Alliance]"
9,111,BEL,"[DeFl, DéFI, FDF]","[Front Démocratique des Francophones, Démocrat...","[Francophone Democratic Front, Christian Democ..."


Further investigations

In [49]:
"""
Investigate, if in the ches meta_df, for each row the combination of country_code and either value in the list of party_name_en or in the list of party_abbrev is unique across all rows, 
i.e. the whole dataframe (such that the combination of either "country_code" and any value in the list "party_abbrev" or any value in the list "party_name_en" identifies a party unambiguously)
"""
tmp_party_name_en = meta_df.explode("party_name_en") # now every row has one english party name
is_unique_country_party_name_en = not tmp_party_name_en.duplicated(["country_code", "party_name_en"]).any()
print("The combination of country code and either value in list of 'party_name_en' is unique across all rows:", is_unique_country_party_name_en)

The combination of country code and either value in list of 'party_name_en' is unique across all rows: False


keep this in mind

In [50]:
# inspect the violations
dups_country_party_name_en = tmp_party_name_en[tmp_party_name_en.duplicated(["country_code", "party_name_en"], keep=False)]
print(dups_country_party_name_en.sort_values(["country_code", "party_name_en"]))


     party_id country_code                            party_abbrev  \
7         109          BEL                        [CD&V, CDV, CVP]   
9         111          BEL                       [DeFl, DéFI, FDF]   
0         102          BEL                                    [PS]   
1         103          BEL  [SP, SP/SPA, SPA, Vooruit, Vooruit/SP]   
369      2609          POL                                [PD, UW]   
371      2612          POL                                    [SD]   

                                            party_name  \
7    [Christelijke Volkspartij, Christen-Democratis...   
9    [Front Démocratique des Francophones, Démocrat...   
0                                   [Parti Socialiste]   
1    [Vooruit, Socialistische Partij, Socialistisch...   
369              [Unia Wolności, Partia Demokratyczna]   
371                        [Stronnictwo Demokratyczne]   

                      party_name_en  
7    Christian Democratic & Flemish  
9    Christian Democratic 

In [51]:
# do the same for the "native" party name
tmp_party_name = meta_df.explode("party_name") # now every row has one party name
is_unique_country_party_name = not tmp_party_name.duplicated(["country_code", "party_name"]).any()
print("The combination of country code and either value in list of 'party_name' is unique across all rows:", is_unique_country_party_name)

The combination of country code and either value in list of 'party_name' is unique across all rows: True


In [52]:
# inspect the violations
dups_country_party_name = tmp_party_name[tmp_party_name.duplicated(["country_code", "party_name"], keep=False)]
print(dups_country_party_name.sort_values(["country_code", "party_name"]))


Empty DataFrame
Columns: [party_id, country_code, party_abbrev, party_name, party_name_en]
Index: []


In [53]:
# do the same for the party abbreviations
tmp_party_abbrev = meta_df.explode("party_abbrev") # now every row has one party abbrev
is_unique_country_party_abbrev = not tmp_party_abbrev.duplicated(["country_code", "party_abbrev"]).any()
print("The combination of country code and either value in list of 'party_abbrev' is unique across all rows:", is_unique_country_party_abbrev)


The combination of country code and either value in list of 'party_abbrev' is unique across all rows: False


In [54]:
# inspect the violations
dups_country_party_abbrev = tmp_party_abbrev[tmp_party_abbrev.duplicated(["country_code", "party_abbrev"], keep=False)]
dups_country_party_abbrev.sort_values(["country_code", "party_abbrev"])
display(dups_country_party_abbrev)

,party_id,country_code,party_abbrev,party_name,party_name_en
139,813,ITA,RI,[Radicali Italiani],[Italian Radicals]
149,825,ITA,RI,[Rinnovamento Italiano],[Italian Renewal–Dini List]
336,2504,LTU,NS,[Naujoji sąjunga (socialliberalai)],[New Union (Social Liberals)]
337,2505,LTU,LLS,"[Lietuvos liberalų sąjunga, Liberalių ir centr...","[Liberal Union of Lithuania, Liberal and Centr..."
344,2512,LTU,LLS,[Lietuvos laisvės sąjunga],[Lithuanian Liberty Union]
359,2527,LTU,NS,[Nacionalinis susivienijimas],[National Alliance]


Now let's also fix the inconsistency found regarding the SLS party in the original (raw) CHES file to prepare it for the merging with migration dataset later on

In [55]:
# load the data
raw_CHES_df = pd.read_csv(PATH_ORIGINAL_CHES_RAW_CSV)
display(raw_CHES_df.head())

,year,country,eastwest,eumember,party_id,party,cmp_id,vote,seat,electionyear,epvote,family,govt,lrgen,lrecon,lrecon_salience,lrecon_dissent,lrecon_blur,galtan,galtan_salience,galtan_dissent,galtan_blur,eu_position,eu_salience,eu_dissent,eu_blur,spendvtax,spendvtax_salience,deregulation,dereg_salience,redistribution,redist_salience,econ_interven,civlib_laworder,civlib_salience,sociallifestyle,social_salience,womens_rights,lgbtq_rights,samesex_marriage,religious_principles,relig_salience,immigrate_policy,immigrate_salience,immigrate_dissent,multiculturalism,multicult_salience,multicult_dissent,nationalism,nationalism_salience,ethnic_minorities,ethnic_salience,urban_rural,urban_salience,environment,enviro_salience,climate_change,climate_change_salience,protectionism,regions,region_salience,international_security,international_salience,us,us_salience,eu_benefit,eu_ep,eu_fiscal,eu_intmark,eu_employ,eu_budgets,eu_agri,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_turkey,eu_russia,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,executive_power,judicial_independence,mip_one,mip_two,mip_three,chesversion
0,1999,1,1,1,102,PS,21322.0,10.2,12.700000,1999,9.59,5,1.0,3.111111,2.625000,NaN,NaN,NaN,3.875000,NaN,NaN,NaN,6.666667,4.722222,1.388889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.333333,6.666667,NaN,6.555555,NaN,NaN,6.625,5.888889,5.666667,6.555555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1,2002,1,1,1,102,PS,21322.0,10.2,12.700000,1999,9.59,5,1.0,3.350000,2.500000,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,6.090000,4.533333,2.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,NaN,4.100000,6.550000,NaN,4.88,5.850,5.000000,6.300000,6.330000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
2,2006,1,1,1,102,PS,21322.0,13.0,16.700001,2003,13.50,5,1.0,3.500000,3.170000,NaN,NaN,NaN,2.830000,NaN,NaN,NaN,5.710000,4.300000,3.430000,NaN,2.670000,6.500000,2.330,6.83,2.000000,8.000000,NaN,3.670000,6.500000,2.330000,5.670000,NaN,NaN,NaN,1.330000,5.670000,2.5,6.330000,NaN,4.20,NaN,NaN,3.17,4.33,3.170000,6.330000,4.400000,3.000000,NaN,NaN,NaN,NaN,NaN,6.170000,7.000000,NaN,NaN,2.67,3.5,NaN,6.500000,NaN,4.000000,NaN,NaN,NaN,6.670,NaN,NaN,5.830000,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
3,2010,1,1,1,102,PS,21322.0,13.7,17.299999,2010,10.88,5,1.0,2.500000,2.571429,NaN,NaN,NaN,2.857143,NaN,NaN,NaN,6.200000,4.523810,2.461539,NaN,1.692308,7.615385,2.375,4.50,1.818182,7.583333,NaN,2.636364,4.727273,1.818182,5.090909,NaN,NaN,NaN,1.454545,2.545455,2.0,6.083333,NaN,3.25,5.545454,NaN,NaN,NaN,2.444444,5.333333,4.000000,2.142857,4.916666,4.833334,NaN,NaN,NaN,5.583334,5.083334,6.090909,3.222222,NaN,NaN,1.133333,6.545455,NaN,5.083333,NaN,NaN,NaN,6.625,NaN,NaN,6.333333,5.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
4,2014,1,1,1,102,PS,21322.0,11.7,15.300000,2014,10.70,5,0.5,2.600000,2.400000,8.4,NaN,NaN,3.400000,3.2,NaN,NaN,6.000000,3.200000,3.000000,NaN,2.400000,NaN,2.000,NaN,1.800000,NaN,1.75,2.200000,NaN,1.600000,NaN,NaN,NaN,NaN,1.000000,NaN,2.8,NaN,NaN,2.00,NaN,NaN,2.25,NaN,2.000000,NaN,3.333333,NaN,3.800000,NaN,NaN,NaN,NaN,6.600000,NaN,5.800000,NaN,NaN,NaN,1.200000,6.400000,NaN,5.000000,NaN,4.2,NaN,6.250,NaN,NaN,6.000000,5.50,NaN,NaN,NaN,NaN,2.6,2.25,NaN,NaN,NaN,13.0,14.0,17.0,2025.1


In [56]:
# quick sanity checks
print(raw_CHES_df["year"].unique())
display(raw_CHES_df.describe())
display(raw_CHES_df.info())
print(raw_CHES_df["party"].dtype)
print(raw_CHES_df["party_id"].dtype)
print(raw_CHES_df["year"].dtype)

[1999 2002 2006 2010 2014 2019 2024]


,year,country,eastwest,eumember,party_id,cmp_id,vote,seat,electionyear,epvote,family,govt,lrgen,lrecon,lrecon_salience,lrecon_dissent,lrecon_blur,galtan,galtan_salience,galtan_dissent,galtan_blur,eu_position,eu_salience,eu_dissent,eu_blur,spendvtax,spendvtax_salience,deregulation,dereg_salience,redistribution,redist_salience,econ_interven,civlib_laworder,civlib_salience,sociallifestyle,social_salience,womens_rights,lgbtq_rights,samesex_marriage,religious_principles,relig_salience,immigrate_policy,immigrate_salience,immigrate_dissent,multiculturalism,multicult_salience,multicult_dissent,nationalism,nationalism_salience,ethnic_minorities,ethnic_salience,urban_rural,urban_salience,environment,enviro_salience,climate_change,climate_change_salience,protectionism,regions,region_salience,international_security,international_salience,us,us_salience,eu_benefit,eu_ep,eu_fiscal,eu_intmark,eu_employ,eu_budgets,eu_agri,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_turkey,eu_russia,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,executive_power,judicial_independence,mip_one,mip_two,mip_three,chesversion
count,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000,1134.000000,1377.000000,1372.000000,1441.000000,1221.000000,1441.000000,1196.000000,1441.000000,1441.000000,737.000000,490.000000,489.000000,1440.000000,737.000000,477.000000,489.000000,1441.000000,1437.000000,1434.000000,490.000000,1124.000000,389.000000,1125.000000,389.000000,1125.000000,879.000000,492.000000,1123.000000,389.000000,880.000000,388.000000,244.000000,244.000000,245.000000,1123.000000,388.000000,1120.000000,879.000000,491.000000,1118.000000,694.000000,246.000000,919.000000,183.000000,1124.000000,387.000000,1110.000000,384.000000,939.000000,695.000000,244.000000,244.000000,485.000000,1118.000000,381.000000,448.000000,203.000000,186.000000,186.000000,448.000000,942.000000,142.000000,1295.000000,309.000000,491.000000,171.000000,1188.000000,312.000000,559.000000,1438.000000,626.000000,243.000000,247.000000,487.000000,492.000000,737.000000,727.000000,246.000000,244.000000,240.000000,231.000000,229.000000,227.000000,1.441000e+03
mean,2012.047883,14.709924,0.621096,0.939625,1480.141568,49473.247795,11.419284,15.073688,2010.706454,11.598835,4.757113,0.292224,5.238517,4.869950,6.592939,2.804542,3.718675,5.139905,6.184942,2.681021,2.965684,4.995091,5.968148,2.435529,3.252559,4.624121,6.180785,4.778908,5.462996,4.413280,6.005743,4.595302,5.169094,6.031098,4.560442,5.596846,4.093361,4.620043,4.332470,4.180368,4.471587,5.380280,5.744692,2.589252,5.446560,5.511637,2.498305,5.241459,5.861366,4.715769,5.762595,4.738113,4.397001,5.093598,4.709384,4.764114,5.193422,5.330244,4.536401,4.876818,4.982199,4.465023,4.494124,5.142688,1.646933,4.813854,4.167364,4.882438,4.551159,3.717981,4.823801,5.434092,5.126692,4.389422,4.404069,3.982621,4.116746,2.638088,3.178144,4.584158,4.748647,4.917749,6.966907,4.934094,3.046536,11.034632,11.720524,12.211454,2.025100e+03
std,8.208976,10.125557,0.485282,0.238263,1011.313545,29681.194882,10.955951,22.728498,8.322277,9.992637,2.657927,0.432099,2.293700,2.146645,1.444620,1.299787,1.572562,2.526470,1.525266,1.505188,1.564218,1.683969,1.652054,1.344984,1.729487,2.179498,1.344363,2.156372,1.406549,2.070705,1.497950,2.165446,2.377283,1.353334,2.718235,1.596485,2.274684,3.027292,3.221962,2.538970,1.875070,2.427580,1.915042,1.392918,2.458558,1.732450,1.283384,2.577354,1.682811,2.522644,1.781669,1.822546,1.611122,2.006017,2.031055,2.371601,1.905846,1.982306,1.749364,1.859748,1.755307,1.438716,2.250450,1.544763,0.691661,1.320140,1.689394,1.483854,1.372200,1.489891,1.267276,1.248324,1.240991,1.535459,1.611815,1.337834,2.550800,1.967790,2.626384,1.951909,2.658010,2.292478,1.753358,2.116088,2.238191,7.352528,9.122488,9.364273,2.274526e-13
min,1999.000000,1.000000,0.000000,0.000000,102.000000,11110.000000,0.000000,0.000000,1996.000000,0.000000,1.000000,0.000000,0.142857,0.090909,2.200000,0.00000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1441 entries, 0 to 1440
Data columns (total 90 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   year                     1441 non-null   int64  
 1   country                  1441 non-null   int64  
 2   eastwest                 1441 non-null   int64  
 3   eumember                 1441 non-null   int64  
 4   party_id                 1441 non-null   int64  
 5   party                    1437 non-null   object 
 6   cmp_id                   1134 non-null   float64
 7   vote                     1377 non-null   float64
 8   seat                     1372 non-null   float64
 9   electionyear             1441 non-null   int64  
 10  epvote                   1221 non-null   float64
 11  family                   1441 non-null   int64  
 12  govt                     1196 non-null   float64
 13  lrgen                    1441 non-null   float64
 14  lrecon                  

None

object
int64
int64


In [57]:
# apparently we are missing the column "expert" (i.e. number of experts who evaluated this party) -> contrary to information in the CHES codebook
#display(raw_CHES_df["expert"].head()) # -> this fails, since column is not present!

In [58]:
# convert column "party_id" and "year" to type string
#raw_CHES_df["party_id"] = raw_CHES_df["party_id"].astype(str)
#raw_CHES_df["year"] = raw_CHES_df["year"].astype(str)
#print(raw_CHES_df["party_id"].dtype)
#print(raw_CHES_df["party_id"].dtype)

**Fix the inconsistency we found for SLS (Slovenian People's Party)**

Slovenian MEP "Franc Bogovič" 's national party affiliations mapped to two parties in CHES at the same time.
Apparently there are two IDs in CHES for the same party SLS (Slovenian People's Party) [see here](https://en.wikipedia.org/wiki/Slovenian_People%27s_Party), namely 2904 and 2919 and also multiple labels ('SLS-SKD', 'SLS', 'SLS-SMS')
but for ID 2919 we only have one datapoint (in 2024) and for ID 2904 we have datapoints on ('02, '06, '10, and '14). therefore merge these two IDs together in CHES

- **CONSEQUENCE** (1) : Party with ID "2919" was removed from CHES_META -> DONE

- **CONSEQUENCE** (2) : Replace ID of party "2919" with ID "2904" in (raw) CHES file to treat the party versions as one -> TO BE DONE

In [59]:
display(raw_CHES_df.loc[raw_CHES_df["party_id"].isin([2904, 2919])])

,year,country,eastwest,eumember,party_id,party,cmp_id,vote,seat,electionyear,epvote,family,govt,lrgen,lrecon,lrecon_salience,lrecon_dissent,lrecon_blur,galtan,galtan_salience,galtan_dissent,galtan_blur,eu_position,eu_salience,eu_dissent,eu_blur,spendvtax,spendvtax_salience,deregulation,dereg_salience,redistribution,redist_salience,econ_interven,civlib_laworder,civlib_salience,sociallifestyle,social_salience,womens_rights,lgbtq_rights,samesex_marriage,religious_principles,relig_salience,immigrate_policy,immigrate_salience,immigrate_dissent,multiculturalism,multicult_salience,multicult_dissent,nationalism,nationalism_salience,ethnic_minorities,ethnic_salience,urban_rural,urban_salience,environment,enviro_salience,climate_change,climate_change_salience,protectionism,regions,region_salience,international_security,international_salience,us,us_salience,eu_benefit,eu_ep,eu_fiscal,eu_intmark,eu_employ,eu_budgets,eu_agri,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_turkey,eu_russia,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,executive_power,judicial_independence,mip_one,mip_two,mip_three,chesversion
1333,2002,29,0,0,2904,SLS-SKD,97620.0,9.60,10.0,2000,NaN,4,1.0,6.800000,5.000000,NaN,NaN,NaN,7.200000,NaN,NaN,NaN,4.800000,6.000000,3.555555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.6,NaN,2.500000,4.67,NaN,5.75,4.67,4.5,4.75,4.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1334,2006,29,0,1,2904,SLS,97521.0,6.80,7.8,2004,8.40,4,1.0,6.400000,5.000000,NaN,NaN,NaN,8.000000,NaN,NaN,NaN,5.000000,4.666667,3.200000,NaN,4.200,4.600,4.600,5.40,4.000000,6.400000,NaN,7.000000,3.6,7.200000,5.200000,NaN,NaN,NaN,6.800000,5.400000,6.200000,5.800000,NaN,6.600000,NaN,NaN,8.0,6.2,5.400000,6.000000,7.200000,8.000000,NaN,NaN,NaN,NaN,NaN,2.400000,6.750,NaN,NaN,4.8,3.8,NaN,4.8,NaN,4.250000,NaN,NaN,NaN,5.60,NaN,NaN,4.500000,4.670000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1335,2010,29,0,1,2904,SLS-SMS,97521.0,5.20,5.6,2008,3.59,4,0.0,6.833333,5.300000,NaN,NaN,NaN,7.833333,NaN,NaN,NaN,4.727273,4.848485,2.111111,NaN,4.375,5.875,4.125,4.25,4.636364,5.888889,NaN,7.000000,4.0,8.083333,6.333333,NaN,NaN,NaN,7.818182,7.545455,7.000000,5.333333,NaN,6.727272,5.3,NaN,NaN,NaN,6.700000,5.444445,7.916667,8.166667,4.666666,3.700000,NaN,NaN,NaN,2.625000,7.125,5.714286,4.333334,NaN,NaN,1.818182,4.5,NaN,4.625000,NaN,NaN,NaN,5.75,NaN,NaN,4.142857,3.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1336,2014,29,0,1,2904,SLS,97521.0,4.00,0.0,2014,8.30,4,0.0,6.461538,6.307693,5.769231,NaN,NaN,7.076923,5.545454,NaN,NaN,6.076923,6.769231,2.454546,NaN,6.100,NaN,6.625,NaN,5.700000,NaN,6.0,6.000000,NaN,7.250000,NaN,NaN,NaN,NaN,7.000000,NaN,6.444445,NaN,NaN,7.125000,NaN,NaN,7.0,NaN,5.625000,NaN,8.222222,NaN,4.800000,NaN,NaN,NaN,NaN,4.181818,NaN,3.000000,NaN,NaN,NaN,1.384615,5.7,NaN,5.545455,NaN,4.818182,NaN,6.10,NaN,NaN,5.666667,3.666667,NaN,NaN,NaN,NaN,5.0,6.875000,NaN,NaN,NaN,18.0,4.0,3.0,2025.1
1367,2024,29,0,1,2919,SLS,NaN,3.41,0.0,2023,7.21,4,NaN,7.923077,6.866667,5.583334,3.333333,5.4,7.500000,6.666666,3.0,6.333334,5.062500,5.133333,4.400000,4.222222,8.000,NaN,6.000,NaN,6.818182,5.454545,NaN,7.333333,NaN,NaN,NaN,6.75,9.2,8.571428,8.000000,NaN,8.133333,7.466667,3.0,8.363636,5.0,NaN,8.5,NaN,7.545455,NaN,9.400000,NaN,5.428571,5.285714,6.8,4.25,6.5,3.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.750000,NaN,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.5,NaN,6.333334,4.0,5.0,5.666666,NaN,6.125,5.75,NaN,NaN,NaN,2025.1


In [60]:
# replace the id '2919' with '2904'
raw_CHES_df["party_id"] = raw_CHES_df["party_id"].replace(to_replace = 2919, value = 2904)
display(raw_CHES_df.loc[raw_CHES_df["party_id"].isin([2904, 2919])])

,year,country,eastwest,eumember,party_id,party,cmp_id,vote,seat,electionyear,epvote,family,govt,lrgen,lrecon,lrecon_salience,lrecon_dissent,lrecon_blur,galtan,galtan_salience,galtan_dissent,galtan_blur,eu_position,eu_salience,eu_dissent,eu_blur,spendvtax,spendvtax_salience,deregulation,dereg_salience,redistribution,redist_salience,econ_interven,civlib_laworder,civlib_salience,sociallifestyle,social_salience,womens_rights,lgbtq_rights,samesex_marriage,religious_principles,relig_salience,immigrate_policy,immigrate_salience,immigrate_dissent,multiculturalism,multicult_salience,multicult_dissent,nationalism,nationalism_salience,ethnic_minorities,ethnic_salience,urban_rural,urban_salience,environment,enviro_salience,climate_change,climate_change_salience,protectionism,regions,region_salience,international_security,international_salience,us,us_salience,eu_benefit,eu_ep,eu_fiscal,eu_intmark,eu_employ,eu_budgets,eu_agri,eu_cohesion,eu_environ,eu_asylum,eu_foreign,eu_turkey,eu_russia,russian_interference,anti_islam_rhetoric,people_vs_elite,antielite_salience,corrupt_salience,members_vs_leadership,executive_power,judicial_independence,mip_one,mip_two,mip_three,chesversion
1333,2002,29,0,0,2904,SLS-SKD,97620.0,9.60,10.0,2000,NaN,4,1.0,6.800000,5.000000,NaN,NaN,NaN,7.200000,NaN,NaN,NaN,4.800000,6.000000,3.555555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.6,NaN,2.500000,4.67,NaN,5.75,4.67,4.5,4.75,4.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1334,2006,29,0,1,2904,SLS,97521.0,6.80,7.8,2004,8.40,4,1.0,6.400000,5.000000,NaN,NaN,NaN,8.000000,NaN,NaN,NaN,5.000000,4.666667,3.200000,NaN,4.200,4.600,4.600,5.40,4.000000,6.400000,NaN,7.000000,3.6,7.200000,5.200000,NaN,NaN,NaN,6.800000,5.400000,6.200000,5.800000,NaN,6.600000,NaN,NaN,8.0,6.2,5.400000,6.000000,7.200000,8.000000,NaN,NaN,NaN,NaN,NaN,2.400000,6.750,NaN,NaN,4.8,3.8,NaN,4.8,NaN,4.250000,NaN,NaN,NaN,5.60,NaN,NaN,4.500000,4.670000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1335,2010,29,0,1,2904,SLS-SMS,97521.0,5.20,5.6,2008,3.59,4,0.0,6.833333,5.300000,NaN,NaN,NaN,7.833333,NaN,NaN,NaN,4.727273,4.848485,2.111111,NaN,4.375,5.875,4.125,4.25,4.636364,5.888889,NaN,7.000000,4.0,8.083333,6.333333,NaN,NaN,NaN,7.818182,7.545455,7.000000,5.333333,NaN,6.727272,5.3,NaN,NaN,NaN,6.700000,5.444445,7.916667,8.166667,4.666666,3.700000,NaN,NaN,NaN,2.625000,7.125,5.714286,4.333334,NaN,NaN,1.818182,4.5,NaN,4.625000,NaN,NaN,NaN,5.75,NaN,NaN,4.142857,3.555556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.1
1336,2014,29,0,1,2904,SLS,97521.0,4.00,0.0,2014,8.30,4,0.0,6.461538,6.307693,5.769231,NaN,NaN,7.076923,5.545454,NaN,NaN,6.076923,6.769231,2.454546,NaN,6.100,NaN,6.625,NaN,5.700000,NaN,6.0,6.000000,NaN,7.250000,NaN,NaN,NaN,NaN,7.000000,NaN,6.444445,NaN,NaN,7.125000,NaN,NaN,7.0,NaN,5.625000,NaN,8.222222,NaN,4.800000,NaN,NaN,NaN,NaN,4.181818,NaN,3.000000,NaN,NaN,NaN,1.384615,5.7,NaN,5.545455,NaN,4.818182,NaN,6.10,NaN,NaN,5.666667,3.666667,NaN,NaN,NaN,NaN,5.0,6.875000,NaN,NaN,NaN,18.0,4.0,3.0,2025.1
1367,2024,29,0,1,2904,SLS,NaN,3.41,0.0,2023,7.21,4,NaN,7.923077,6.866667,5.583334,3.333333,5.4,7.500000,6.666666,3.0,6.333334,5.062500,5.133333,4.400000,4.222222,8.000,NaN,6.000,NaN,6.818182,5.454545,NaN,7.333333,NaN,NaN,NaN,6.75,9.2,8.571428,8.000000,NaN,8.133333,7.466667,3.0,8.363636,5.0,NaN,8.5,NaN,7.545455,NaN,9.400000,NaN,5.428571,5.285714,6.8,4.25,6.5,3.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.750000,NaN,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.5,NaN,6.333334,4.0,5.0,5.666666,NaN,6.125,5.75,NaN,NaN,NaN,2025.1


In [ ]:
# now save this also to parquet
#raw_CHES_df.to_parquet(PATH_INTERMED_CHES_50_CHES_RAW, engine='fastparquet')